question 1: 

Task: use 17 industry portfolio as regressor, estimate HML and MOM 

1. create HML and MOM using 6 portfolios
2. the returns of HML is factor return of value portoflio; the returns of MOM is factor return of momentum portfolio

*portfolio of each year seems to be constructed based on data at the end of June of the same year, so do the same for this?

For each of HML and MOM: 
1. regress 17 industry portfolio on HML/MOM

2. run one of each type of regression: linear (forward and hybrid), ridge, LASSO, and random forest 
2a. Hybird on MOM gave same model set as forward, and we already know the two variable one has lowest CV SSE, so two variable is the best pick for MOM Linear, so i didn't write code for hybrid again

3. pick best 2


Compare selection criteria across models and across (1a), (1b), and (1c)
1. For selection of optimal model with the same number of predictors we only use in-sample SSE
2. For overall selection we use CV SSE, AIC, and BIC (in slide)

Summary (haven't done Random Forest for any): 

1a. HML: Linear ['Trans', 'Machn', 'Steel', 'Finan', 'Other'] (best), Lasso (second best); MOM: Ridge (best), LASSO (second best)

1b. HML: Linear ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars'] (best), Lasso (second best); MOM: Lasso (best), Ridge (second best)

1c. HML: Linear ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines'] (best), Lasso (second best); MOM: Lasso (best), Ridge (second best)
  

select three best models for HML and for MOM separately for question 1 

  

question 2: 

Task: use 17 industry portfolio as regressor, Comment on the ease or difficulty of replicating the components as opposed to the entire return 

1. use three best model for HML and compare that with accuracy of SV+BV and SG+BG 

2. use three best model for MOM and compare that with accuracy of SH+BH and SL+BL 

  

question 3: 

Task: combine the three procedures, and test accuracy on HML and MOM vs each of the component 

if still have word count, can test accuracy on long-side and short side and/or do pairs combinations of three procedures 


In [1]:
import numpy as np
import pandas as pd
import itertools
from itertools import combinations
import calendar
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from collections import Counter
import math
from sklearn.model_selection import PredefinedSplit

In [ ]:
mom = pd.read_csv("factor_return_mom.csv")

hml = pd.read_csv("factor_return_value.csv")

#The 6 Portfolios Formed on Size and Momentum (contains SH BH SL BL)
size_momentum = pd.read_csv("6_Portfolios_size_mom.csv")
size_momentum = size_momentum.iloc[:1174]
size_momentum['Unnamed: 0'] = size_momentum['Unnamed: 0'].astype(int)

#The 6 Portfolios Formed on Size and Value (contains SV BV SG BG)
size_value = pd.read_csv("6_Portfolios_size_value.csv")
#small lo seems to be small growth

#The 17 Industry Portfolios (regressors)
seventeen = pd.read_csv("17_Industry_Portfolios_cleaned.csv")

# Display the first few rows of the DataFrame
print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

   Unnamed: 0   HML
0      192607 -2.43
1      192608  3.82
2      192609  0.13
3      192610  0.70
4      192611 -0.51
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
   Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
0      192607      1.0874   0.9081     -0.0695    5.7168   1.8971    2.0066
1      192608

In [ ]:
#make everything start at 192701
hml = hml.iloc[6:]
size_value = size_value.iloc[6:]
seventeen = seventeen.iloc[6:]

print(hml.head())
print(mom.head())
print(size_momentum.head())
print(size_value.head())
print(seventeen.head())

# check if times are the same, or some dataset is missing certain dates

    Unnamed: 0   HML
6       192701  4.54
7       192702  2.94
8       192703 -2.61
9       192704  0.81
10      192705  4.73
   Unnamed: 0    MOM
0      192701   0.36
1      192702  -2.14
2      192703   3.61
3      192704   4.30
4      192705  30.00
   Unnamed: 0  SMALL LoPRIOR  ME1 PRIOR2  SMALL HiPRIOR  BIG LoPRIO  \
0      192701           0.08        3.57           0.40       -0.40   
1      192702           7.01        6.48           5.51        7.27   
2      192703          -3.46       -2.88          -2.06       -3.57   
3      192704          -0.75       -0.48           3.32       -2.72   
4      192705           2.48       11.15           6.83        5.52   

   ME2 PRIOR2  B HiPRIOR  
0        0.26       0.00  
1        3.80       4.47  
2       -0.18       2.26  
3        0.75       1.81  
4        4.96       7.18  
    Unnamed: 0  SMALL LoBM  ME1 BM2  SMALL HiBM  BIG LoBM  ME2 BM2  BIG HiBM
6       192701     -0.7991  -0.3782      3.9316   -0.2621   0.0321    4.0885
7    

In [4]:
size_momentum['Unnamed: 0'] = pd.to_datetime(size_momentum['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
size_value['Unnamed: 0'] = pd.to_datetime(size_value['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
hml['Unnamed: 0'] = pd.to_datetime(hml['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
mom['Unnamed: 0'] = pd.to_datetime(mom['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')
seventeen['Unnamed: 0'] = pd.to_datetime(seventeen['Unnamed: 0'].astype(str), format='%Y%m').dt.to_period('M')

In [5]:
print(seventeen.columns)
seventeen.columns = seventeen.columns.str.strip() #strip spaces
predictors = seventeen.columns[1:]

Index(['Unnamed: 0', 'Food ', 'Mines', 'Oil  ', 'Clths', 'Durbl', 'Chems',
       'Cnsum', 'Cnstr', 'Steel', 'FabPr', 'Machn', 'Cars ', 'Trans', 'Utils',
       'Rtail', 'Finan', 'Other'],
      dtype='object')


In [6]:
# Set date as the index
seventeen.set_index('Unnamed: 0', inplace=True)
size_momentum.set_index('Unnamed: 0', inplace=True)
size_value.set_index('Unnamed: 0', inplace=True)
hml.set_index('Unnamed: 0', inplace=True)
mom.set_index('Unnamed: 0', inplace=True)

In [7]:
#group data into 5-year intervals
seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
size_value_grouped = size_value.groupby(pd.Grouper(freq='5Y'))
mom_grouped = mom.groupby(pd.Grouper(freq='5Y'))
hml_grouped = hml.groupby(pd.Grouper(freq='5Y'))

/tmp/ipykernel_5550/903688633.py:2: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_5550/903688633.py:2: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  seventeen_grouped = seventeen.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_5550/903688633.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_5550/903688633.py:3: FutureWarning: Resampling with a PeriodIndex is deprecated. Cast index to DatetimeIndex before resampling instead.
  size_momentum_grouped = size_momentum.groupby(pd.Grouper(freq='5Y'))
/tmp/ipykernel_5550/903688633.py:4: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  size_value_grouped = si

In [8]:
group_labels = list(seventeen_grouped.groups.keys())

In [9]:
print(group_labels)

[Period('1927', '5Y-DEC'), Period('1932', '5Y-DEC'), Period('1937', '5Y-DEC'), Period('1942', '5Y-DEC'), Period('1947', '5Y-DEC'), Period('1952', '5Y-DEC'), Period('1957', '5Y-DEC'), Period('1962', '5Y-DEC'), Period('1967', '5Y-DEC'), Period('1972', '5Y-DEC'), Period('1977', '5Y-DEC'), Period('1982', '5Y-DEC'), Period('1987', '5Y-DEC'), Period('1992', '5Y-DEC'), Period('1997', '5Y-DEC'), Period('2002', '5Y-DEC'), Period('2007', '5Y-DEC'), Period('2012', '5Y-DEC'), Period('2017', '5Y-DEC'), Period('2022', '5Y-DEC')]


# Analysis 1

## 5-year data

### Linear regression

In [10]:
# linear (variable selection)

linear = LinearRegression(fit_intercept=False)

#### forward stepwise selection

In [42]:
###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/3533222164.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


/tmp/ipykernel_1548/3533222164.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/3533222164.py:18: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/3533222164.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [43]:
###### two variable

# first selected variable is Trans for both

#remove from predictors
predictors1 = predictors[~predictors.isin(['Trans'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors1:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/35664418.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1548/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/35664418.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/35664418.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


In [44]:
###### three variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)

#remove from predictors
predictorsh = predictors1[~predictors1.isin(['Machn'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/3751845197.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Steel
The MOM predictor with the lowest SSE is: Mines


/tmp/ipykernel_1548/3751845197.py:43: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [45]:
###### four variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Steel'])]
predictorsm = predictorsm[~predictorsm.isin(['Mines'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2373838400.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1548/2373838400.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/2373838400.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/2373838400.py:44: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [46]:
###### five variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/613824281.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Rtail


/tmp/ipykernel_1548/613824281.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/613824281.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [47]:
###### six variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Other'])]
predictorsm = predictorsm[~predictorsm.isin(['Rtail'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2402805268.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Oil
The MOM predictor with the lowest SSE is: FabPr


/tmp/ipykernel_1548/2402805268.py:47: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [48]:
###### seven variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Oil'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2660957166.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: Clths


/tmp/ipykernel_1548/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/2660957166.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [49]:
###### eight variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/4154706649.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Chems


In [50]:
###### nine variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Chems'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2104319118.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Utils
The MOM predictor with the lowest SSE is: Food


/tmp/ipykernel_1548/2104319118.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/2104319118.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/2104319118.py:53: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [51]:
###### ten variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Utils'])]
predictorsm = predictorsm[~predictorsm.isin(['Food'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2135745534.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Machn


/tmp/ipykernel_1548/2135745534.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [52]:
###### 11 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cnsum'])]
predictorsm = predictorsm[~predictorsm.isin(['Machn'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/1372259047.py:42: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths
The MOM predictor with the lowest SSE is: Steel


/tmp/ipykernel_1548/1372259047.py:59: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [53]:
###### 12 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Clths'])]
predictorsm = predictorsm[~predictorsm.isin(['Steel'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1548/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2863303116.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr
The MOM predictor with the lowest SSE is: Oil


In [54]:
###### 13 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cnstr'])]
predictorsm = predictorsm[~predictorsm.isin(['Oil'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1548/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/412403489.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Cnsum


/tmp/ipykernel_1548/412403489.py:65: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1548/412403489.py:65: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [55]:
###### 14 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Cnsum'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2396443549.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: Utils


In [56]:
###### 15 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['Utils'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars', 'Durbl']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum', 'Utils']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/14378800.py:54: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Food
The MOM predictor with the lowest SSE is: Cars


In [57]:
###### 16 variable

# first selected variable is Trans for both
# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)
# 15 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils + Cars (MOM)

#remove from predictors
predictorsh = predictorsh[~predictorsh.isin(['Food'])]
predictorsm = predictorsm[~predictorsm.isin(['Cars'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 
                                     'Utils', 'Cnsum', 'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food']]
    predict_hml = pd.Series()
    #iterate through 5-year intervals
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 
                                     'Food', 'Machn', 'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])

    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1548/2679972848.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2679972848.py:57: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2679972848.py:74: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Mines
The MOM predictor with the lowest SSE is: Durbl


In [14]:
# Best Models
best_model_hml = []
best_model_mom = []

# first selected variable is Trans for both
best_model_hml.append('Trans')
best_model_mom.append('Trans')

# two variable model is Trans + Machn (HML), Trans + Cnstr (MOM)
best_model_hml.append(['Trans','Machn'])
best_model_mom.append(['Trans','Cnstr'])

# three variable model is Trans + Machn + Steel (HML), Trans + Cnstr + Mines (MOM)
best_model_hml.append(['Trans','Machn', 'Steel'])
best_model_mom.append(['Trans','Cnstr', 'Mines'])
# four variable model is Trans + Machn + Steel + Finan (HML), Trans + Cnstr + Mines + Finan (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan'])
# five variable model is Trans + Machn + Steel + Finan + Other (HML), \
# Trans + Cnstr + Mines + Finan + Rtail (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail'])
# six variable model is Trans + Machn + Steel + Finan + Other + Oil (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr'])
# seven variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths'])
# eight variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems'])
# nine variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food'])
# 10 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn'])
# 11 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel'])
# 12 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil'])
# 13 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum'])
# 14 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + Steel + Oil + Cnsum + Utils (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils'])
# 15 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + \
# Steel + Oil + Cnsum + Utils + Cars (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars'])
# 16 variable model is Trans + Machn + Steel + Finan + Other + Oil + Chems + Rtail + Utils + Cnsum + \
# Clths + Cnstr + Cars + Durbl + Food + Mines (HML), \
# Trans + Cnstr + Mines + Finan + Rtail + FabPr + Clths + Chems + Food + Machn + \
# Steel + Oil + Cnsum + Utils + Cars + Durbl (MOM)
best_model_hml.append(['Trans','Machn', 'Steel', 'Finan', 'Other', 'Oil', 'Chems', 'Rtail', 'Utils', 'Cnsum', 
                       'Clths', 'Cnstr', 'Cars', 'Durbl', 'Food', 'Mines'])
best_model_mom.append(['Trans','Cnstr', 'Mines', 'Finan', 'Rtail', 'FabPr', 'Clths', 'Chems', 'Food', 'Machn', 
                       'Steel', 'Oil', 'Cnsum', 'Utils', 'Cars', 'Durbl'])
#17
best_model_hml.append(predictors.tolist())
best_model_mom.append(predictors.tolist())

In [15]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
            train = explanatory.get_group(group_labels[i]).values
            test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in best_model_mom:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
             train = explanatory.get_group(group_labels[i]).values
             test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate MOM using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

hml_to_exclude = hml_grouped.get_group(group_labels[0])
mom_to_exclude = mom_grouped.get_group(group_labels[0])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude.index)]
hml_5yr_actual = hml_5yr_actual.values
mom_5yr_actual = mom[~mom.index.isin(mom_to_exclude.index)]
mom_5yr_actual = mom_5yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)
n_mom = len(mom_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

/tmp/ipykernel_5550/1772011360.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/1772011360.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/1772011360.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [175]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")
min_aic = aic_mom.index(min(aic_mom))
best_mom = best_model_mom[min_aic]
print(f"The MOM predictor with the lowest AIC is: {best_mom}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")
min_bic = bic_mom.index(min(bic_mom))
best_mom = best_model_mom[min_bic]
print(f"The MOM predictor with the lowest BIC is: {best_mom}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest SSE is: ['Trans', 'Cnstr']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest AIC is: ['Trans', 'Cnstr']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The MOM predictor with the lowest BIC is: ['Trans', 'Cnstr']


#### hybrid stepwise selection

In [61]:
### hybrid stepwise selection

# use the optimal model got from forward

# 5 to 4 

#store sse
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM

mom_pred = pd.Series(['Trans', 'Cnstr'])

for pred in mom_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_mom = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")


/tmp/ipykernel_1548/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1548/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2367577185.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/2367577185.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [62]:
# 4 to 3

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan'])

for pred in hml_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

# MOM will wait for final model selection

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1548/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1548/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/3752236646.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Machn


In [63]:
# 3 to 2

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Steel', 'Finan'])

for pred in hml_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

# MOM will wait for final model selection

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Steel


/tmp/ipykernel_1548/988068310.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/988068310.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/988068310.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [64]:
# 2 to 1

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Finan'])

for pred in hml_pred:
    #remove one variable
    col = mom_pred[~mom_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels)):
        train = explanatory.get_group(group_labels[i]).values

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

# MOM will wait for final model selection

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1548/3282136415.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1548/3282136415.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Finan


In [65]:
#choose best model using hybrid stepwise selection and out of sample SSE, AIC, BIC

best_model_hml = ['Trans', ['Trans', 'Finan'], ['Trans', 'Steel', 'Finan'], ['Trans', 'Machn', 'Steel', 'Finan'], 
                  ['Trans', 'Machn', 'Steel', 'Finan', 'Other']]

#store predicted value
predict_hml_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels)-1):
        if isinstance(pred, list):
            train = explanatory.get_group(group_labels[i]).values
            test = explanatory.get_group(group_labels[i+1]).values
        else:
            train = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)

        #estimate HML using 5 year data
        linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

hml_to_exclude = hml_grouped.get_group(group_labels[0])
#actual response
hml_5yr_actual = hml[~hml.index.isin(hml_to_exclude.index)]
hml_5yr_actual = hml_5yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_5yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

/tmp/ipykernel_1548/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1548/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1548/611564430.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [66]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']


In [67]:
### backward stepwise selection

### Ridge

In [55]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-1):
        train = seventeen_grouped.get_group(group_labels[i]).values
        test = seventeen_grouped.get_group(group_labels[i+1]).values
        
        #estimate HML using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
        
        #predict next 5 year HML
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
        #estimate MOM using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    
        #predict next 5 year MOM
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_ridge_a = float(best_hml)
print(f"The alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_ridge_a = float(best_mom)
print(f"The alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_5550/3120643919.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/3120643919.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/3120643919.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The alpha with the lowest SSE is: 100.0
The alpha with the lowest SSE is: 1000.0


/tmp/ipykernel_5550/3120643919.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/3120643919.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


### LASSO

In [56]:
### lasso

# we can perhaps look at the variables chosen by LASSO if we want to talk about and iterpret them

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-1):
        train = seventeen_grouped.get_group(group_labels[i]).values
        test = seventeen_grouped.get_group(group_labels[i+1]).values
        
        #estimate HML using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
        
        #predict next 5 year HML
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
        #estimate MOM using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    
        #predict next 5 year MOM
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_lasso_a = float(best_hml)
print(f"The alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_lasso_a = float(best_mom)
print(f"The alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_5550/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2867682998.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/home/codespace/.local/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.009e+02,

/tmp/ipykernel_5550/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2867682998.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The alpha with the lowest SSE is: 0.31622776601683794
The alpha with the lowest SSE is: 10.0


/tmp/ipykernel_5550/2867682998.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2867682998.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


### Random Forest

In [46]:
#random forest regression

test_fold = [-1] * 60 + [0] * 60
cv_splitter = PredefinedSplit(test_fold)

#set seed to 42
rf_model = RandomForestRegressor(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
    'max_features': ['sqrt'] #slide 7 said use sqrt p predictors where p is the total number of predictors
}

# Time series cross-validator
# tscv = TimeSeriesSplit(n_splits=5) #this way the cross validation only happens in chronological order

#create model
rf_hml = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=cv_splitter, 
                        n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
rf_mom = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=cv_splitter, 
                        n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)

# tune hyperparameters

param_hml = []
param_mom = []

for i in range(0,len(group_labels)-2): #
    x_train = seventeen_grouped.get_group(group_labels[i]).values
    hml_train = hml_grouped.get_group(group_labels[i]).values
    mom_train = mom_grouped.get_group(group_labels[i]).values

    x_test = seventeen_grouped.get_group(group_labels[i+1]).values
    hml_test = hml_grouped.get_group(group_labels[i+1]).values
    mom_test = mom_grouped.get_group(group_labels[i+1]).values

    x = np.concatenate((x_train, x_test), axis=0)
    h = np.concatenate((hml_train, hml_test), axis=0)
    m = np.concatenate((mom_train, mom_test), axis=0)

    rf_hml.fit(x, h)
    param_hml.append(rf_hml.best_params_)

    rf_mom.fit(x, m)
    param_mom.append(rf_mom.best_params_)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

In [47]:
print("For HML")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_hml]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_hml]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_hml]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_hml]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

#not going to run this again, too much time:
# For HML
# The most commonly appeared value for 'n_estimators' is 50 with 9 occurrences.
# The most commonly appeared value for 'min_samples_split' is 2 with 10 occurrences.
# The most commonly appeared value for 'min_samples_leaf' is 1 with 10 occurrences.
# The most commonly appeared value for 'max_depth' is None with 8 occurrences.

For HML
The most commonly appeared value for 'n_estimators' is 50 with 9 occurrences.
The most commonly appeared value for 'min_samples_split' is 2 with 10 occurrences.
The most commonly appeared value for 'min_samples_leaf' is 1 with 10 occurrences.
The most commonly appeared value for 'max_depth' is None with 8 occurrences.


In [49]:
print("For MOM")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_mom]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_mom]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_mom]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_mom]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

# For MOM
# The most commonly appeared value for 'n_estimators' is 50 with 9 occurrences.
# The most commonly appeared value for 'min_samples_split' is 10 with 10 occurrences.
# The most commonly appeared value for 'min_samples_leaf' is 4 with 11 occurrences.
# The most commonly appeared value for 'max_depth' is 20 with 12 occurrences.

For MOM
The most commonly appeared value for 'n_estimators' is 50 with 9 occurrences.
The most commonly appeared value for 'min_samples_split' is 10 with 10 occurrences.
The most commonly appeared value for 'min_samples_leaf' is 4 with 11 occurrences.
The most commonly appeared value for 'max_depth' is 20 with 12 occurrences.


In [57]:
#overall selection

# forward & hybird selection: The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other']
### The MOM predictor with the lowest SSE is: ['Trans', 'Cnstr']

#choose best model using hybrid stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df5 = pd.DataFrame()
predict_mom_df5 = pd.DataFrame()

#HML
predict_hml = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Machn', 'Steel', 'Finan', 'Other']]
for i in range(0,len(group_labels)-1):
    train = explanatory.get_group(group_labels[i]).values
    test = explanatory.get_group(group_labels[i+1]).values

    #estimate HML using 5 year data
    linear.fit(train,hml_grouped.get_group(group_labels[i]).values)
    #predict next 5 year HML
    predict = pd.Series(linear.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])
    
predict_hml_df5['Linear'] = predict_hml

#MOM
predict_mom = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Cnstr']]
for i in range(0,len(group_labels)-1):
    train = explanatory.get_group(group_labels[i]).values
    test = explanatory.get_group(group_labels[i+1]).values

    #estimate MOM using 5 year data
    linear.fit(train,mom_grouped.get_group(group_labels[i]).values)
    #predict next 5 year MOM
    predict = pd.Series(linear.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])
    
predict_mom_df5['Linear'] = predict_mom

# ridge: HML alpha = 100 (stored in hml_ridge_a), MOM alpha = 1000 (stored in mom_ridge_a)

#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate HML using 5 year data
    ridge_model = Ridge(alpha=hml_ridge_a, fit_intercept=False)
    ridge_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
    hml_coef_ridge = ridge_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df5['Ridge'] = predict_hml
    
#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate MOM using 5 year data
    ridge_model = Ridge(alpha=mom_ridge_a, fit_intercept=False)
    ridge_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    mom_coef_ridge = ridge_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df5['Ridge'] = predict_mom

# lasso: HML alpha = 0.31622776601683794 (stored in hml_lasso_a), MOM alpha = 10 (stored in mom_lasso_a)

#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate HML using 5 year data
    lasso_model = Lasso(alpha=hml_lasso_a, fit_intercept=False)
    lasso_model.fit(train, hml_grouped.get_group(group_labels[i]).values)
    hml_coef_lasso = lasso_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df5['LASSO'] = predict_hml

#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-1):
    train = seventeen_grouped.get_group(group_labels[i]).values
    test = seventeen_grouped.get_group(group_labels[i+1]).values
        
    #estimate MOM using 5 year data
    lasso_model = Lasso(alpha=mom_lasso_a, fit_intercept=False)
    lasso_model.fit(train, mom_grouped.get_group(group_labels[i]).values)
    mom_coef_lasso = lasso_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df5['LASSO'] = predict_mom

# random forest

#HML

random_pred_hml = pd.Series()

rf_model_hml = RandomForestRegressor(n_estimators=50, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                                 random_state=42)

for i in range(0,len(group_labels)-1): #
    x_train = seventeen_grouped.get_group(group_labels[i]).values
    hml_train = hml_grouped.get_group(group_labels[i]).values

    x_test = seventeen_grouped.get_group(group_labels[i+1]).values

    rf_model_hml.fit(x_train, hml_train)

    #predict next 5 year HML
    predict = pd.Series(rf_model_hml.predict(x_test).flatten())
    random_pred_hml = pd.concat([random_pred_hml, predict])

#MOM

random_pred_mom = pd.Series()

rf_model_mom = RandomForestRegressor(n_estimators=50, max_depth=20, min_samples_split=10, min_samples_leaf=4,
                                 random_state=42)

for i in range(0,len(group_labels)-1): #
    x_train = seventeen_grouped.get_group(group_labels[i]).values
    mom_train = mom_grouped.get_group(group_labels[i]).values

    x_test = seventeen_grouped.get_group(group_labels[i+1]).values

    rf_model_mom.fit(x_train, mom_train)

    #predict next 5 year MOM
    predict = pd.Series(rf_model_mom.predict(x_test).flatten())
    random_pred_mom = pd.concat([random_pred_mom, predict])

# pick best 2

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df5 - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_rf_hml = ((random_pred_hml - hml_5yr_actual.flatten()) ** 2).sum()
sse_mom = ((predict_mom_df5 - mom_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_rf_mom = ((random_pred_mom - mom_5yr_actual.flatten()) ** 2).sum()

# Compute SST
hml_mean = hml_5yr_actual.mean()
sst_hml = ((hml_5yr_actual - hml_mean) ** 2).sum()

mom_mean = mom_5yr_actual.mean()
sst_mom = ((mom_5yr_actual - mom_mean) ** 2).sum()

# calculate R^2
r2_hml = 1 - (sse_hml / sst_hml)
r2_rf_hml = 1 - (sse_rf_hml / sst_hml)
r2_mom = 1 - (sse_mom / sst_mom)
r2_rf_mom = 1 - (sse_rf_mom / sst_mom)

/tmp/ipykernel_5550/4253247860.py:23: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/4253247860.py:38: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/4253247860.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [58]:
# count num of non zero coef
k_ridge_hml = np.count_nonzero(hml_coef_ridge)
k_ridge_mom = np.count_nonzero(mom_coef_ridge)

k_lasso_hml = np.count_nonzero(hml_coef_lasso)
k_lasso_mom = np.count_nonzero(mom_coef_lasso)

k_l_hml = 5
k_l_mom = 2

k_hml = [k_l_hml, k_ridge_hml, k_lasso_hml]

k_mom = [k_l_mom, k_ridge_mom, k_lasso_mom]

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_hml)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_hml)]
aic_hml = pd.Series(aic_hml)
bic_hml = pd.Series(bic_hml)
aic_hml.index = sse_hml.index
bic_hml.index = sse_hml.index

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_mom)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_mom)]
aic_mom = pd.Series(aic_mom)
bic_mom = pd.Series(bic_mom)
aic_mom.index = sse_mom.index
bic_mom.index = sse_mom.index

In [27]:
#sort in ascending order
sse_hml = pd.concat([sse_hml, pd.Series(sse_rf_hml)]).sort_values()
sse_mom = pd.concat([sse_mom, pd.Series(sse_rf_mom)]).sort_values()
aic_hml = aic_hml.sort_values()
aic_mom = aic_mom.sort_values()
bic_hml = bic_hml.sort_values()
bic_mom = bic_mom.sort_values()
r2_hml = pd.concat([r2_hml, pd.Series(r2_rf_hml)]).sort_values(ascending=False)
r2_mom = pd.concat([r2_mom, pd.Series(r2_rf_mom)]).sort_values(ascending=False)

In [28]:
print(r2_hml)
print(bic_hml)
# so based on this 5-year best for HML are Linear and Ridge

Linear    0.506163
Ridge     0.456258
LASSO     0.455052
0         0.219043
0         0.219043
dtype: float64
Linear    1.871038
LASSO     2.019906
Ridge     2.042881
dtype: float64


In [29]:
print(r2_mom)
print(bic_mom)

# so based on this 5-year best for MOM are Linear and Ridge

Linear    0.142543
Ridge     0.090334
LASSO     0.059479
0         0.053024
0         0.053024
dtype: float64
Linear    6.390208
LASSO     6.482671
Ridge     6.543780
dtype: float64


In [166]:
# Choose optimal model (minimum SSE)
# best_hml = sse_hml.idxmin()
# print(f"The HML predictor with the lowest SSE is: {best_hml}")
# second = sse_hml.index[1]
# print(f"The HML predictor with the second lowest SSE is: {second}")
# Choose optimal model (max R2)
# best_hml = r2_hml.idxmax()
# print(f"The HML predictor with the highest R2 is: {best_hml}")
# second = r2_hml.index[2]
# print(f"The HML predictor with the second highest R2 is: {second}")
# Choose optimal model (minimum AIC)
# best_hml = aic_hml.idxmin()
# print(f"The HML predictor with the lowest AIC is: {best_hml}")
# second = aic_hml.index[1]
# print(f"The HML predictor with the second lowest AIC is: {second}")
# Choose optimal model (minimum BIC)
# best_hml = bic_hml.idxmin()
# print(f"The HML predictor with the lowest BIC is: {best_hml}")
# second = bic_hml.index[1]
# print(f"The HML predictor with the second lowest BIC is: {second}")

# Choose optimal model (minimum SSE)
# best_mom = sse_mom.idxmin()
# print(f"The MOM predictor with the lowest SSE is: {best_mom}")
# second = sse_mom.index[1]
# print(f"The MOM predictor with the second lowest SSE is: {second}")
# Choose optimal model (max R2)
# best_mom = r2_mom.idxmax()
# print(f"The MOM predictor with the highest R2 is: {best_mom}")
# second = r2_mom.index[2]
# print(f"The MOM predictor with the second highest R2 is: {second}")
# Choose optimal model (minimum AIC)
# best_mom = aic_mom.idxmin()
# print(f"The MOM predictor with the lowest AIC is: {best_mom}")
# second = aic_mom.index[1]
# print(f"The MOM predictor with the second lowest AIC is: {second}")
# Choose optimal model (minimum BIC)
# best_mom = bic_mom.idxmin()
# print(f"The MOM predictor with the lowest BIC is: {best_mom}")
# second = bic_mom.index[1]
# print(f"The MOM predictor with the second lowest BIC is: {second}")

#0 is random forest

The HML predictor with the lowest SSE is: Linear
The HML predictor with the second lowest SSE is: Ridge
The HML predictor with the highest R2 is: Linear
The HML predictor with the second highest R2 is: Ridge
The HML predictor with the lowest AIC is: Linear
The HML predictor with the second lowest AIC is: LASSO
The HML predictor with the lowest BIC is: Linear
The HML predictor with the second lowest BIC is: LASSO
The MOM predictor with the lowest SSE is: Linear
The MOM predictor with the second lowest SSE is: Ridge
The MOM predictor with the highest R2 is: Linear
The MOM predictor with the second highest R2 is: Ridge
The MOM predictor with the lowest AIC is: Linear
The MOM predictor with the second lowest AIC is: Ridge
The MOM predictor with the lowest BIC is: Linear
The MOM predictor with the second lowest BIC is: LASSO


In [59]:
five_sse_hml = ((predict_hml_df5[180:] - hml_5yr_actual[180:].reshape(-1, 1)) ** 2).sum(axis=0)
five_sse_mom = ((predict_mom_df5[180:] - mom_5yr_actual[180:].reshape(-1, 1)) ** 2).sum(axis=0)

# Compute SST
five_hml_mean = hml_5yr_actual[180:].mean()
five_sst_hml = ((hml_5yr_actual[180:] - five_hml_mean) ** 2).sum()

five_mom_mean = mom_5yr_actual[180:].mean()
five_sst_mom = ((mom_5yr_actual[180:] - five_mom_mean) ** 2).sum()

# calculate R^2
five_r2_hml = 1 - (five_sse_hml / five_sst_hml)
five_r2_mom = 1 - (five_sse_mom / five_sst_mom)

In [60]:
# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(five_sse_hml, k_hml)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(five_sse_hml, k_hml)]
aic_hml = pd.Series(aic_hml)
bic_hml = pd.Series(bic_hml)
aic_hml.index = five_sse_hml.index
bic_hml.index = five_sse_hml.index

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(five_sse_mom, k_mom)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(five_sse_mom, k_mom)]
aic_mom = pd.Series(aic_mom)
bic_mom = pd.Series(bic_mom)
aic_mom.index = five_sse_mom.index
bic_mom.index = five_sse_mom.index

In [61]:
#get values of the two best methods so I don't have to calculate them again :P

# five_sse_hml_linear = five_sse_hml.loc['Linear']
# five_sse_hml_lasso = five_sse_hml.loc['Ridge']

# five_aic_hml_linear = aic_hml.loc['Linear']
# five_aic_hml_lasso = aic_hml.loc['LASSO']

five_bic_hml_linear = bic_hml.loc['Linear']
five_bic_hml_ridge = bic_hml.loc['Ridge']

five_r2_hml_linear = five_r2_hml.loc['Linear']
five_r2_hml_ridge = five_r2_hml.loc['Ridge']

# five_sse_mom_ridge = five_sse_mom.loc['Ridge']
# five_sse_mom_linear = five_sse_mom.loc['Linear']

# five_aic_mom_ridge = aic_mom.loc['Ridge']
# five_aic_mom_linear = aic_mom.loc['Linear']

five_bic_mom_ridge = bic_mom.loc['Ridge']
five_bic_mom_linear = bic_mom.loc['Linear']

five_r2_mom_ridge = five_r2_mom.loc['Ridge']
five_r2_mom_linear = five_r2_mom.loc['Linear']

## 10-year data

### Linear Regression

In [82]:
###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
        train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1937/3889845230.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/3889845230.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1937/3889845230.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


/tmp/ipykernel_1937/3889845230.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/3889845230.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1937/3889845230.py:24: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [282]:
###### two variable

predictors1 = predictors[~predictors.isin(['Trans'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1207218035.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1207218035.py:35: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1207218035.py:26: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [283]:
###### 3 variable

predictorsh = predictors1[~predictors1.isin(['Machn'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2062258075.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Steel
The MOM predictor with the lowest SSE is: Mines


In [284]:
###### 4 variable

predictorsh = predictorsh[~predictorsh.isin(['Steel'])]
predictorsm = predictorsm[~predictorsm.isin(['Mines'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3282065233.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3282065233.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3282065233.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Clths


/tmp/ipykernel_1917/3282065233.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3282065233.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3282065233.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [285]:
###### 5 variable

predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2550458330.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/2550458330.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2550458330.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2550458330.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [286]:
###### 6 variable

predictorsh = predictorsh[~predictorsh.isin(['Other'])]
predictorsm = predictorsm[~predictorsm.isin(['Steel'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1363685795.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1917/1363685795.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [287]:
###### 7 variable

predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3856651286.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Other


/tmp/ipykernel_1917/3856651286.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [288]:
###### 8 variable

predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/669673142.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/669673142.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/669673142.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: Cnsum


In [289]:
###### 9 variable

predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['Cnsum'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2643745856.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Oil
The MOM predictor with the lowest SSE is: Oil


In [290]:
###### 10 variable

predictorsh = predictorsh[~predictorsh.isin(['Oil'])]
predictorsm = predictorsm[~predictorsm.isin(['Oil'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/930407204.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/930407204.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/930407204.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Utils
The MOM predictor with the lowest SSE is: Rtail


/tmp/ipykernel_1917/930407204.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/930407204.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [291]:
###### 11 variable

predictorsh = predictorsh[~predictorsh.isin(['Utils'])]
predictorsm = predictorsm[~predictorsm.isin(['Rtail'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2546659055.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: Cars


/tmp/ipykernel_1917/2546659055.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [292]:
###### 12 variable

predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['Cars'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2287680030.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2287680030.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2287680030.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Durbl


/tmp/ipykernel_1917/2287680030.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [293]:
###### 13 variable

predictorsh = predictorsh[~predictorsh.isin(['Cnsum'])]
predictorsm = predictorsm[~predictorsm.isin(['Durbl'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/4044487405.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4044487405.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4044487405.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Food
The MOM predictor with the lowest SSE is: Chems


In [294]:
###### 14 variable

predictorsh = predictorsh[~predictorsh.isin(['Food'])]
predictorsm = predictorsm[~predictorsm.isin(['Chems'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum', 'Food']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl', 'Chems']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1735760812.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1735760812.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1735760812.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr
The MOM predictor with the lowest SSE is: FabPr


In [295]:
###### 15 variable

predictorsh = predictorsh[~predictorsh.isin(['Cnstr'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum', 'Food', 'Cnstr']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl', 'Chems', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3876624578.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3876624578.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3876624578.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: FabPr
The MOM predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/3876624578.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3876624578.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3876624578.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [296]:
###### 16 variable

predictorsh = predictorsh[~predictorsh.isin(['FabPr'])]
predictorsm = predictorsm[~predictorsm.isin(['Machn'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 
                                     'Oil', 'Utils', 'Durbl', 'Cnsum', 'Food', 'Cnstr', 'FabPr']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 
                                     'Oil', 'Rtail', 'Cars', 'Durbl', 'Chems', 'FabPr', 'Machn']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2326514894.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2326514894.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2326514894.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2326514894.py:50: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths
The MOM predictor with the lowest SSE is: Utils


In [20]:
# best models

best_model_hml = []
best_model_mom = []

# 1 var
best_model_hml.append('Trans')
best_model_mom.append('Trans')

# 2 var
best_model_hml.append(['Trans', 'Machn'])
best_model_mom.append(['Trans', 'Cnstr'])

# 3 var
best_model_hml.append(['Trans', 'Machn', 'Steel'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines'])

# 4 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths'])

# 5 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel'])

# 6 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan'])

# 7 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other'])

# 8 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum'])

# 9 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil'])

# 10 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail'])

# 11 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars'])

# 12 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl'])

# 13 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems'])

# 14 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food', 'Cnstr'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems', 'FabPr'])

# 15 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food', 'Cnstr', 'FabPr'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems', 'FabPr', 'Machn'])

# 16 var
best_model_hml.append(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars', 'Rtail', 'Chems', 'Oil', 'Utils', 
                       'Durbl', 'Cnsum', 'Food', 'Cnstr', 'FabPr', 'Clths'])
best_model_mom.append(['Trans', 'Cnstr', 'Mines', 'Clths', 'Steel', 'Finan', 'Other', 'Cnsum', 'Oil', 'Rtail', 
                       'Cars', 'Durbl', 'Chems', 'FabPr', 'Machn', 'Utils'])

#17
best_model_hml.append(predictors.tolist())
best_model_mom.append(predictors.tolist())

In [33]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    # since it's 5-year rolling scheme
    for i in range(0,len(group_labels)-2):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            test = explanatory.get_group(group_labels[i+2]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)
        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in best_model_mom:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-2):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            test = explanatory.get_group(group_labels[i+2]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)
        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

hml_to_exclude1 = hml_grouped.get_group(group_labels[0])
mom_to_exclude1 = mom_grouped.get_group(group_labels[0])
hml_to_exclude2 = hml_grouped.get_group(group_labels[1])
mom_to_exclude2 = mom_grouped.get_group(group_labels[1])
#actual response
hml_10yr_actual = hml[~hml.index.isin(hml_to_exclude1.index)]
hml_10yr_actual = hml_10yr_actual[~hml_10yr_actual.index.isin(hml_to_exclude2.index)]
hml_10yr_actual = hml_10yr_actual.values
mom_10yr_actual = mom[~mom.index.isin(mom_to_exclude1.index)]
mom_10yr_actual = mom_10yr_actual[~mom_10yr_actual.index.isin(mom_to_exclude2.index)]
mom_10yr_actual = mom_10yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_10yr_actual)
n_mom = len(mom_10yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

/tmp/ipykernel_5550/2921649573.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2921649573.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2921649573.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [172]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")
min_aic = aic_mom.index(min(aic_mom))
best_mom = best_model_mom[min_aic]
print(f"The MOM predictor with the lowest AIC is: {best_mom}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")
min_bic = bic_mom.index(min(bic_mom))
best_mom = best_model_mom[min_bic]
print(f"The MOM predictor with the lowest BIC is: {best_mom}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The MOM predictor with the lowest SSE is: Trans
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The MOM predictor with the lowest AIC is: Trans
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The MOM predictor with the lowest BIC is: Trans


### Hybird

In [300]:
#hybrid

# can only do it on HML since MOM is only one predictor

### hybrid stepwise selection

# use the optimal model got from forward

# 6 to 5

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Cars


/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3022032707.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [301]:
# 5 to 4

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Steel', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3104293146.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [302]:
# 4 to 3

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/185918700.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [303]:
# 3 to 2

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3657427776.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3657427776.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3657427776.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan


In [304]:
# 2 to 1

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 2):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Other


/tmp/ipykernel_1917/2328425969.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2328425969.py:29: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [86]:
#choose best model using hybrid stepwise selection and out of sample SSE, AIC, BIC

best_model_hml = ['Trans', ['Trans', 'Other'], ['Trans', 'Other', 'Finan'], ['Trans', 'Machn', 'Finan', 'Other'], 
                  ['Trans', 'Machn', 'Steel', 'Finan', 'Other'], ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]

#store predicted value
predict_hml_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    # since it's 5-year rolling scheme
    for i in range(0,len(group_labels)-2):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            test = explanatory.get_group(group_labels[i+2]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2), axis=0)
        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

k_values = list(range(1, 7))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

/tmp/ipykernel_1937/3867542640.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/3867542640.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/3867542640.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [87]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']


### Ridge

In [34]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-2):
        train1 = seventeen_grouped.get_group(group_labels[i]).values
        train2 = seventeen_grouped.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        test = seventeen_grouped.get_group(group_labels[i+2]).values

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
        
        #estimate HML using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, y)
        
        #predict next 5 year HML
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
    
        #estimate MOM using 5 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, y)
    
        #predict next 5 year MOM
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_ridge_a = float(best_hml)
print(f"The HML alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_ridge_a = float(best_mom)
print(f"The MOM alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_5550/1016213838.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/1016213838.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/1016213838.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML alpha with the lowest SSE is: 100.0
The MOM alpha with the lowest SSE is: 10000.0


### LASSO

In [35]:
# ridge and lasso (tune alpha)

### lasso

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-2):
        train1 = seventeen_grouped.get_group(group_labels[i]).values
        train2 = seventeen_grouped.get_group(group_labels[i+1]).values
        train = np.concatenate((train1, train2), axis=0)

        test = seventeen_grouped.get_group(group_labels[i+2]).values

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
        
        #estimate HML using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, y)
        
        #predict next 5 year HML
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y = np.concatenate((y1, y2), axis=0)
    
        #estimate MOM using 5 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, y)
    
        #predict next 5 year MOM
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_lasso_a10 = float(best_hml)
print(f"The HML alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_lasso_a10 = float(best_mom)
print(f"The MOM alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_5550/2405914250.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2405914250.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/2405914250.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML alpha with the lowest SSE is: 0.1
The MOM alpha with the lowest SSE is: 316.2277660168379


/tmp/ipykernel_5550/2405914250.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/2405914250.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/2405914250.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

### Random Forest

In [78]:
#random forest regression

test_fold = [-1] * 120 + [0] * 60
cv_splitter = PredefinedSplit(test_fold)

#set seed to 42
rf_model = RandomForestRegressor(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
    'max_features': ['sqrt'] #slide 7 said use sqrt p predictors where p is the total number of predictors
}

# Time series cross-validator
# tscv = TimeSeriesSplit(n_splits=5) #this way the cross validation only happens in chronological order

#create model
rf_hml = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=cv_splitter, 
                        n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
rf_mom = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=cv_splitter, 
                        n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)

# tune hyperparameters

param_hml = []
param_mom = []

for i in range(0,len(group_labels)-3): #
    x_train1 = seventeen_grouped.get_group(group_labels[i]).values
    x_train2 = seventeen_grouped.get_group(group_labels[i+1]).values

    hml_train1 = hml_grouped.get_group(group_labels[i]).values
    hml_train2 = hml_grouped.get_group(group_labels[i+1]).values
    
    mom_train1 = mom_grouped.get_group(group_labels[i]).values
    mom_train2 = mom_grouped.get_group(group_labels[i+1]).values

    x_test = seventeen_grouped.get_group(group_labels[i+2]).values
    hml_test = hml_grouped.get_group(group_labels[i+2]).values
    mom_test = mom_grouped.get_group(group_labels[i+2]).values

    x = np.concatenate((x_train1, x_train2, x_test), axis=0)
    h = np.concatenate((hml_train1, hml_train2, hml_test), axis=0)
    m = np.concatenate((mom_train1, mom_train2, mom_test), axis=0)

    rf_hml.fit(x, h)
    param_hml.append(rf_hml.best_params_)

    rf_mom.fit(x, m)
    param_mom.append(rf_mom.best_params_)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

In [79]:
print("For HML")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_hml]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_hml]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_hml]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_hml]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

# For HML
# The most commonly appeared value for 'n_estimators' is 50 with 13 occurrences.
# The most commonly appeared value for 'min_samples_split' is 2 with 9 occurrences.
# The most commonly appeared value for 'min_samples_leaf' is 1 with 10 occurrences.
# The most commonly appeared value for 'max_depth' is 10 with 7 occurrences.

For HML
The most commonly appeared value for 'n_estimators' is 50 with 13 occurrences.
The most commonly appeared value for 'min_samples_split' is 2 with 9 occurrences.
The most commonly appeared value for 'min_samples_leaf' is 1 with 10 occurrences.
The most commonly appeared value for 'max_depth' is 10 with 7 occurrences.


In [80]:
print("For MOM")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_mom]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_mom]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_mom]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_mom]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

# For MOM
# The most commonly appeared value for 'n_estimators' is 50 with 9 occurrences.
# The most commonly appeared value for 'min_samples_split' is 10 with 8 occurrences.
# The most commonly appeared value for 'min_samples_leaf' is 4 with 9 occurrences.
# The most commonly appeared value for 'max_depth' is 20 with 7 occurrences.

For MOM
The most commonly appeared value for 'n_estimators' is 50 with 9 occurrences.
The most commonly appeared value for 'min_samples_split' is 10 with 8 occurrences.
The most commonly appeared value for 'min_samples_leaf' is 4 with 9 occurrences.
The most commonly appeared value for 'max_depth' is 20 with 7 occurrences.


In [36]:
#overall

#store predicted value
predict_hml_df10 = pd.DataFrame()
predict_mom_df10 = pd.DataFrame()

#linear (forward & hybrid)
# The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']

predict_hml = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]
for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate HML using 5 year data
    linear.fit(train,y)
    #predict next 5 year HML
    predict = pd.Series(linear.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])
    
predict_hml_df10['Linear'] = predict_hml

# The MOM predictor with the lowest SSE is: Trans
predict_mom = pd.Series()
explanatory = seventeen_grouped['Trans']
for i in range(0,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
    train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate MOM using 10 year data
    linear.fit(train,y)
    #predict next 5 year MOM
    predict = pd.Series(linear.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])
    
predict_mom_df10['Linear'] = predict_mom

#ridge
# HML alpha: 100.0 (stored in hml_ridge_a), MOM alpha: 10000.0 (stored in mom_ridge_a)
#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+2]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate HML using 10 year data
    ridge_model = Ridge(alpha=hml_ridge_a, fit_intercept=False)
    ridge_model.fit(train, y)
    hml_coef_ridge = ridge_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df10['Ridge'] = predict_hml

#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+2]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate MOM using 10 year data
    ridge_model = Ridge(alpha=mom_ridge_a, fit_intercept=False)
    ridge_model.fit(train, y)
    mom_coef_ridge = ridge_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df10['Ridge'] = predict_mom

#lasso
# HML alpha: 0.1 (stored in hml_lasso_a), MOM alpha: 316.2277660168379 (stored in mom_lasso_a)

#HML
predict_hml = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+2]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate HML using 5 year data
    lasso_model = Lasso(alpha=hml_lasso_a10, fit_intercept=False)
    lasso_model.fit(train, y)
        
    #predict next 5 year HML
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df10['Lasso'] = predict_hml

#MOM
predict_mom = pd.Series()

for i in range(0,len(group_labels)-2):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+2]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y = np.concatenate((y1, y2), axis=0)

    #estimate MOM using 5 year data
    lasso_model = Lasso(alpha=mom_lasso_a10, fit_intercept=False)
    lasso_model.fit(train, y)
        
    #predict next 5 year MOM
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df10['Lasso'] = predict_mom

# random forest
#HML

random_pred_hml = pd.Series()

rf_model_hml = RandomForestRegressor(n_estimators=50, max_depth=10, min_samples_split=2, min_samples_leaf=1,
                                 random_state=42)

for i in range(0,len(group_labels)-2): #
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    x_train = np.concatenate((train1, train2), axis=0)

    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    hml_train = np.concatenate((y1, y2), axis=0)

    x_test = seventeen_grouped.get_group(group_labels[i+2]).values

    rf_model_hml.fit(x_train, hml_train)

    #predict next 5 year HML
    predict = pd.Series(rf_model_hml.predict(x_test).flatten())
    random_pred_hml = pd.concat([random_pred_hml, predict])

#MOM

random_pred_mom = pd.Series()

rf_model_mom = RandomForestRegressor(n_estimators=50, max_depth=20, min_samples_split=10, min_samples_leaf=4,
                                 random_state=42)

for i in range(0,len(group_labels)-2): #
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    x_train = np.concatenate((train1, train2), axis=0)

    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    mom_train = np.concatenate((y1, y2), axis=0)

    x_test = seventeen_grouped.get_group(group_labels[i+2]).values

    rf_model_mom.fit(x_train, mom_train)

    #predict next 5 year MOM
    predict = pd.Series(rf_model_mom.predict(x_test).flatten())
    random_pred_mom = pd.concat([random_pred_mom, predict])

# pick best 2

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df10 - hml_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_rf_hml = ((random_pred_hml - hml_10yr_actual.flatten()) ** 2).sum()
sse_mom = ((predict_mom_df10 - mom_10yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_rf_mom = ((random_pred_mom - mom_10yr_actual.flatten()) ** 2).sum()

# Compute SST
hml_mean = hml_10yr_actual.mean()
sst_hml = ((hml_10yr_actual - hml_mean) ** 2).sum()

mom_mean = mom_10yr_actual.mean()
sst_mom = ((mom_10yr_actual - mom_mean) ** 2).sum()

# calculate R^2
r2_hml = 1 - (sse_hml / sst_hml)
r2_rf_hml = 1 - (sse_rf_hml / sst_hml)
r2_mom = 1 - (sse_mom / sst_mom)
r2_rf_mom = 1 - (sse_rf_mom / sst_mom)

/tmp/ipykernel_5550/3425433647.py:27: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/3425433647.py:49: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/3425433647.py:76: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [37]:
# count num of non zero coef
k_ridge_hml = np.count_nonzero(hml_coef_ridge)
k_ridge_mom = np.count_nonzero(mom_coef_ridge)

k_lasso_hml = np.count_nonzero(hml_coef_lasso)
k_lasso_mom = np.count_nonzero(mom_coef_lasso)

k_l_hml = 6
k_l_mom = 1

k_hml = [k_l_hml, k_ridge_hml, k_lasso_hml]

k_mom = [k_l_mom, k_ridge_mom, k_lasso_mom]

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_hml)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_hml)]
aic_hml = pd.Series(aic_hml)
bic_hml = pd.Series(bic_hml)
aic_hml.index = sse_hml.index
bic_hml.index = sse_hml.index

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_mom)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_mom)]
aic_mom = pd.Series(aic_mom)
bic_mom = pd.Series(bic_mom)
aic_mom.index = sse_mom.index
bic_mom.index = sse_mom.index

In [39]:
#sort in ascending order
sse_hml = pd.concat([sse_hml, pd.Series(sse_rf_hml)]).sort_values()
sse_mom = pd.concat([sse_mom, pd.Series(sse_rf_mom)]).sort_values()
aic_hml = aic_hml.sort_values()
aic_mom = aic_mom.sort_values()
bic_hml = bic_hml.sort_values()
bic_mom = bic_mom.sort_values()
r2_hml = pd.concat([r2_hml, pd.Series(r2_rf_hml)]).sort_values(ascending=False)
r2_mom = pd.concat([r2_mom, pd.Series(r2_rf_mom)]).sort_values(ascending=False)

In [40]:
print(r2_hml)
print(bic_hml)

# Lasso 1+2
# Ridge 2+3
# Linear 3+1
# RF 4. we see R2 of RF is singnificantly lower compared to the other three, so for tracking accuracy purpose 
# we do not select RF even though its total score ties with Linear
# based on this best two for 10 year HML are Lasso and Linear

Lasso     0.460541
Ridge     0.459803
Linear    0.456209
0         0.186516
0         0.186516
dtype: float64
Linear    1.593238
Lasso     1.631466
Ridge     1.659247
dtype: float64


In [41]:
print(r2_mom)
print(bic_mom)

# Lasso 1+1
# RF 2
# Ridge 3+3
# Linear 4+2

# based on this the best two for 10 year MOM are Lasso and RF

Lasso     0.071447
0         0.023420
0         0.023420
Ridge     0.001398
Linear   -0.047279
dtype: float64
Lasso     5.515089
Linear    5.628809
Ridge     5.686875
dtype: float64


In [180]:
# Choose optimal model (minimum SSE)
# best_hml = sse_hml.idxmin()
# print(f"The HML predictor with the lowest SSE is: {best_hml}")
# second = sse_hml.index[1]
# print(f"The HML predictor with the second lowest SSE is: {second}")
# Choose optimal model (max R2)
# best_hml = r2_hml.idxmax()
# print(f"The HML predictor with the highest R2 is: {best_hml}")
# second = r2_hml.index[2]
# print(f"The HML predictor with the second highest R2 is: {second}")
# Choose optimal model (minimum AIC)
# best_hml = aic_hml.idxmin()
# print(f"The HML predictor with the lowest AIC is: {best_hml}")
# second = aic_hml.index[1]
# print(f"The HML predictor with the second lowest AIC is: {second}")
# Choose optimal model (minimum BIC)
# best_hml = bic_hml.idxmin()
# print(f"The HML predictor with the lowest BIC is: {best_hml}")
# second = bic_hml.index[1]
# print(f"The HML predictor with the second lowest BIC is: {second}")

# Choose optimal model (minimum SSE)
# best_mom = sse_mom.idxmin()
# print(f"The MOM predictor with the lowest SSE is: {best_mom}")
# second = sse_mom.index[1]
# print(f"The MOM predictor with the second lowest SSE is: {second}")
# Choose optimal model (max R2)
# best_mom = r2_mom.idxmax()
# print(f"The MOM predictor with the highest R2 is: {best_mom}")
# second = r2_mom.index[2]
# print(f"The MOM predictor with the second highest R2 is: {second}")
# Choose optimal model (minimum AIC)
# best_mom = aic_mom.idxmin()
# print(f"The MOM predictor with the lowest AIC is: {best_mom}")
# second = aic_mom.index[1]
# print(f"The MOM predictor with the second lowest AIC is: {second}")
# Choose optimal model (minimum BIC)
# best_mom = bic_mom.idxmin()
# print(f"The MOM predictor with the lowest BIC is: {best_mom}")
# second = bic_mom.index[1]
# print(f"The MOM predictor with the second lowest BIC is: {second}")

The HML predictor with the lowest SSE is: Lasso
The HML predictor with the second lowest SSE is: Ridge
The HML predictor with the highest R2 is: Lasso
The HML predictor with the second highest R2 is: Ridge
The HML predictor with the lowest AIC is: Linear
The HML predictor with the second lowest AIC is: Lasso
The HML predictor with the lowest BIC is: Linear
The HML predictor with the second lowest BIC is: Lasso
The MOM predictor with the lowest SSE is: Lasso
The MOM predictor with the second lowest SSE is: 0
The MOM predictor with the highest R2 is: Lasso
The MOM predictor with the second highest R2 is: 0
The MOM predictor with the lowest AIC is: Lasso
The MOM predictor with the second lowest AIC is: Ridge
The MOM predictor with the lowest BIC is: Lasso
The MOM predictor with the second lowest BIC is: Linear


In [42]:
ten_sse_hml = ((predict_hml_df10[120:] - hml_10yr_actual[120:].reshape(-1, 1)) ** 2).sum(axis=0)
ten_sse_mom = ((predict_mom_df10[120:] - mom_10yr_actual[120:].reshape(-1, 1)) ** 2).sum(axis=0)
ten_sse_rf_mom = ((random_pred_mom[120:] - mom_10yr_actual[120:].flatten()) ** 2).sum()

# Compute SST
ten_hml_mean = hml_10yr_actual[120:].mean()
ten_sst_hml = ((hml_10yr_actual[120:] - ten_hml_mean) ** 2).sum()

ten_mom_mean = mom_10yr_actual[120:].mean()
ten_sst_mom = ((mom_10yr_actual[120:] - ten_mom_mean) ** 2).sum()

# calculate R^2
ten_r2_hml = 1 - (ten_sse_hml / ten_sst_hml)
ten_r2_mom = 1 - (ten_sse_mom / ten_sst_mom)
ten_r2_rf_mom = 1 - (ten_sse_rf_mom / ten_sst_mom)

In [43]:
# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(ten_sse_hml, k_hml)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(ten_sse_hml, k_hml)]
aic_hml = pd.Series(aic_hml)
bic_hml = pd.Series(bic_hml)
aic_hml.index = ten_sse_hml.index
bic_hml.index = ten_sse_hml.index

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(ten_sse_mom, k_mom)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(ten_sse_mom, k_mom)]
aic_mom = pd.Series(aic_mom)
bic_mom = pd.Series(bic_mom)
aic_mom.index = ten_sse_mom.index
bic_mom.index = ten_sse_mom.index

In [44]:
#get values of the two best methods so I don't have to calculate them again :P

# ten_sse_hml_linear = ten_sse_hml.loc['Linear']
# ten_sse_hml_lasso = ten_sse_hml.loc['Lasso']

# ten_aic_hml_linear = aic_hml.loc['Linear']
# ten_aic_hml_lasso = aic_hml.loc['Lasso']

ten_bic_hml_linear = bic_hml.loc['Linear']
ten_bic_hml_lasso = bic_hml.loc['Lasso']

ten_r2_hml_linear = ten_r2_hml.loc['Linear']
ten_r2_hml_lasso = ten_r2_hml.loc['Lasso']

# ten_sse_mom_random_forest = ten_sse_rf_mom
# ten_sse_mom_lasso = ten_sse_mom.loc['Lasso']

# ten_aic_mom_lasso = aic_mom.loc['Lasso']

ten_bic_mom_lasso = bic_mom.loc['Lasso']

ten_r2_mom_random_forest = ten_r2_rf_mom
ten_r2_mom_lasso = ten_r2_mom.loc['Lasso']

## 20-year data

### Linear Regression

#### Forward Stepwise Selection

In [312]:
###### one variable

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
        train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
        train3 = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
        train4 = explanatory.get_group(group_labels[i+3]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/4080392235.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4080392235.py:39: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/4080392235.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Trans
The MOM predictor with the lowest SSE is: Trans


In [313]:
###### two variable

predictors1 = predictors[~predictors.isin(['Trans'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for pred in predictors:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans']]
    predict_hml = pd.Series()
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1684246663.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1684246663.py:41: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1684246663.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Machn
The MOM predictor with the lowest SSE is: Cnstr


/tmp/ipykernel_1917/1684246663.py:30: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1684246663.py:41: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [314]:
###### 3 variable

predictorsh = predictors1[~predictors1.isin(['Machn'])]
predictorsm = predictors1[~predictors1.isin(['Cnstr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3037319448.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3037319448.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3037319448.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan
The MOM predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/3037319448.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3037319448.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3037319448.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [315]:
###### 4 variable

predictorsh = predictorsh[~predictorsh.isin(['Finan'])]
predictorsm = predictorsm[~predictorsm.isin(['Steel'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1930612126.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1930612126.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1930612126.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Other
The MOM predictor with the lowest SSE is: Finan


/tmp/ipykernel_1917/1930612126.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1930612126.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/1930612126.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [316]:
###### 5 variable

predictorsh = predictorsh[~predictorsh.isin(['Other'])]
predictorsm = predictorsm[~predictorsm.isin(['Finan'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3583753146.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3583753146.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3583753146.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Steel
The MOM predictor with the lowest SSE is: Rtail


/tmp/ipykernel_1917/3583753146.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [317]:
###### 6 variable

predictorsh = predictorsh[~predictorsh.isin(['Steel'])]
predictorsm = predictorsm[~predictorsm.isin(['Rtail'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#HML
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 5 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2632246840.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2632246840.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2632246840.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cars
The MOM predictor with the lowest SSE is: Clths


In [318]:
###### 7 variable

predictorsh = predictorsh[~predictorsh.isin(['Cars'])]
predictorsm = predictorsm[~predictorsm.isin(['Clths'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2559275411.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2559275411.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2559275411.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2559275411.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2559275411.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Utils
The MOM predictor with the lowest SSE is: Utils


In [319]:
###### 8 variable

predictorsh = predictorsh[~predictorsh.isin(['Utils'])]
predictorsm = predictorsm[~predictorsm.isin(['Utils'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3355288708.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3355288708.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3355288708.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3355288708.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Chems
The MOM predictor with the lowest SSE is: Oil


In [320]:
###### 8 variable

predictorsh = predictorsh[~predictorsh.isin(['Chems'])]
predictorsm = predictorsm[~predictorsm.isin(['Oil'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/89472585.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/89472585.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/89472585.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


The HML predictor with the lowest SSE is: Oil
The MOM predictor with the lowest SSE is: Mines


/tmp/ipykernel_1917/89472585.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/89472585.py:56: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


In [321]:
###### 10 variable

predictorsh = predictorsh[~predictorsh.isin(['Oil'])]
predictorsm = predictorsm[~predictorsm.isin(['Mines'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3062641659.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3062641659.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Rtail
The MOM predictor with the lowest SSE is: Other


/tmp/ipykernel_1917/3062641659.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3062641659.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3062641659.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [322]:
###### 11 variable

predictorsh = predictorsh[~predictorsh.isin(['Rtail'])]
predictorsm = predictorsm[~predictorsm.isin(['Other'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil',
                                     'Rtail']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines',
                                     'Other']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Durbl
The MOM predictor with the lowest SSE is: FabPr


/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3537046991.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [323]:
###### 12 variable

predictorsh = predictorsh[~predictorsh.isin(['Durbl'])]
predictorsm = predictorsm[~predictorsm.isin(['FabPr'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil',
                                     'Rtail', 'Durbl']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines',
                                     'Other', 'FabPr']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/2259889641.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2259889641.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2259889641.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2259889641.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2259889641.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr
The MOM predictor with the lowest SSE is: Food


/tmp/ipykernel_1917/2259889641.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2259889641.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/2259889641.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [324]:
###### 13 variable

predictorsh = predictorsh[~predictorsh.isin(['Cnstr'])]
predictorsm = predictorsm[~predictorsm.isin(['Food'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil',
                                     'Rtail', 'Durbl', 'Cnstr']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines',
                                     'Other', 'FabPr', 'Food']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/1812032724.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1812032724.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1812032724.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum
The MOM predictor with the lowest SSE is: Durbl


In [325]:
###### 14 variable

predictorsh = predictorsh[~predictorsh.isin(['Cnsum'])]
predictorsm = predictorsm[~predictorsm.isin(['Durbl'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil',
                                     'Rtail', 'Durbl', 'Cnstr', 'Cnsum']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines',
                                     'Other', 'FabPr', 'Food', 'Durbl']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/3174929298.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3174929298.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3174929298.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths
The MOM predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/3174929298.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3174929298.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/3174929298.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [326]:
###### 15 variable

predictorsh = predictorsh[~predictorsh.isin(['Clths'])]
predictorsm = predictorsm[~predictorsm.isin(['Machn'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil',
                                     'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines',
                                     'Other', 'FabPr', 'Food', 'Durbl', 'Machn']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

/tmp/ipykernel_1917/827849316.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/827849316.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: FabPr
The MOM predictor with the lowest SSE is: Cars


/tmp/ipykernel_1917/827849316.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/827849316.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_1917/827849316.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [327]:
###### 16 variable

predictorsh = predictorsh[~predictorsh.isin(['FabPr'])]
predictorsm = predictorsm[~predictorsm.isin(['Cars'])]

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in predictorsh:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil',
                                     'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr']]
    predict_hml = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in predictorsm:
    #choose one variable
    explanatory = seventeen_grouped[[pred, 'Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines',
                                     'Other', 'FabPr', 'Food', 'Durbl', 'Machn', 'Cars']]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)
mom_hml = ((predict_mom_df - mom['MOM'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = mom_hml.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

The HML predictor with the lowest SSE is: Mines
The MOM predictor with the lowest SSE is: Chems


/tmp/ipykernel_1917/3578488847.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3578488847.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3578488847.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [45]:
# best models

best_model_hml = []
best_model_mom = []

# 1 var
best_model_hml.append('Trans')
best_model_mom.append('Trans')

# 2 var
best_model_hml.append(['Trans', 'Machn'])
best_model_mom.append(['Trans', 'Cnstr'])

# 3 var
best_model_hml.append(['Trans', 'Machn', 'Finan'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel'])

# 4 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan'])

# 5 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail'])

# 6 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths'])

# 7 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils'])

# 8 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil'])

# 9 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines'])

# 10 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other'])

# 11 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other',
                       'FabPr'])

# 12 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other',
                       'FabPr', 'Food'])

# 13 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other',
                       'FabPr', 'Food', 'Durbl'])

# 14 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum', 'Clths'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other',
                       'FabPr', 'Food', 'Durbl', 'Machn'])

# 15 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other',
                       'FabPr', 'Food', 'Durbl', 'Machn', 'Cars'])

# 16 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines'])
best_model_mom.append(['Trans', 'Cnstr', 'Steel', 'Finan', 'Rtail', 'Clths', 'Utils', 'Oil', 'Mines', 'Other',
                       'FabPr', 'Food', 'Durbl', 'Machn', 'Cars', 'Chems'])

#17
best_model_hml.append(predictors.tolist())
best_model_mom.append(predictors.tolist())

In [46]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    # since it's 5-year rolling scheme
    for i in range(0,len(group_labels)-4):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            train3 = explanatory.get_group(group_labels[i+2]).values
            train4 = explanatory.get_group(group_labels[i+3]).values
            test = explanatory.get_group(group_labels[i+4]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            train3 = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
            train4 = explanatory.get_group(group_labels[i+3]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+4]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#MOM
for pred in best_model_mom:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_mom = pd.Series()
    for i in range(0,len(group_labels)-4):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            train3 = explanatory.get_group(group_labels[i+2]).values
            train4 = explanatory.get_group(group_labels[i+3]).values
            test = explanatory.get_group(group_labels[i+4]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            train3 = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
            train4 = explanatory.get_group(group_labels[i+3]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+4]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate MOM using 20 year data
        linear.fit(train,y)
        #predict next 5 year MOM
        predict = pd.Series(linear.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_mom_df[f'{pred}'] = predict_mom

hml_to_exclude1 = hml_grouped.get_group(group_labels[0])
mom_to_exclude1 = mom_grouped.get_group(group_labels[0])
hml_to_exclude2 = hml_grouped.get_group(group_labels[1])
mom_to_exclude2 = mom_grouped.get_group(group_labels[1])
hml_to_exclude3 = hml_grouped.get_group(group_labels[2])
mom_to_exclude3 = mom_grouped.get_group(group_labels[2])
hml_to_exclude4 = hml_grouped.get_group(group_labels[3])
mom_to_exclude4 = mom_grouped.get_group(group_labels[3])
#actual response
hml_20yr_actual = hml[~hml.index.isin(hml_to_exclude1.index)]
hml_20yr_actual = hml_20yr_actual[~hml_20yr_actual.index.isin(hml_to_exclude2.index)]
hml_20yr_actual = hml_20yr_actual[~hml_20yr_actual.index.isin(hml_to_exclude3.index)]
hml_20yr_actual = hml_20yr_actual[~hml_20yr_actual.index.isin(hml_to_exclude4.index)]
hml_20yr_actual = hml_20yr_actual.values

mom_20yr_actual = mom[~mom.index.isin(mom_to_exclude1.index)]
mom_20yr_actual = mom_20yr_actual[~mom_20yr_actual.index.isin(mom_to_exclude2.index)]
mom_20yr_actual = mom_20yr_actual[~mom_20yr_actual.index.isin(mom_to_exclude3.index)]
mom_20yr_actual = mom_20yr_actual[~mom_20yr_actual.index.isin(mom_to_exclude4.index)]
mom_20yr_actual = mom_20yr_actual.values

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Number of observations in the actual data
n_hml = len(hml_20yr_actual)
n_mom = len(mom_20yr_actual)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_values)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_values)]

/tmp/ipykernel_5550/4065939363.py:38: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/4065939363.py:38: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/4065939363.py:38: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [200]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
print(f"The MOM predictor with the lowest SSE is: {best_mom}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")
min_aic = aic_mom.index(min(aic_mom))
best_mom = best_model_mom[min_aic]
print(f"The MOM predictor with the lowest AIC is: {best_mom}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")
min_bic = bic_mom.index(min(bic_mom))
best_mom = best_model_mom[min_bic]
print(f"The MOM predictor with the lowest BIC is: {best_mom}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines']
The MOM predictor with the lowest SSE is: Trans
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines']
The MOM predictor with the lowest AIC is: Trans
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils']
The MOM predictor with the lowest BIC is: Trans


#### Hybrid Stepwise Selection

In [331]:
# can only do it on HML since MOM is only one predictor

### hybrid stepwise selection

# use the optimal model got from forward

# 16 to 15

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML predictor with the lowest SSE is: Mines


/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/960325784.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [332]:
# can only do it on HML since MOM is only one predictor

### hybrid stepwise selection

# use the optimal model got from forward

# 15 to 14

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr', 'Cnsum', 'Clths', 'FabPr'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: FabPr


/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2181652096.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [333]:
# 14 to 13

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr', 'Cnsum', 'Clths'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3848692771.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/3848692771.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3848692771.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3848692771.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Clths


/tmp/ipykernel_1917/3848692771.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3848692771.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [334]:
# 13 to 12

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr', 'Cnsum'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3177782914.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3177782914.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3177782914.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Chems


In [335]:
# 12 to 11

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr', 'Cnsum'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3367030882.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3367030882.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3367030882.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnsum


/tmp/ipykernel_1917/3367030882.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [336]:
# 11 to 10

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3094983666.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3094983666.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3094983666.py:34: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Utils


In [337]:
# 10 to 9

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil', 'Rtail', 'Durbl', 'Cnstr'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3520737075.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3520737075.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3520737075.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Rtail


/tmp/ipykernel_1917/3520737075.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3520737075.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3520737075.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [338]:
# 9 to 8

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil', 'Durbl', 'Cnstr'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3000088127.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3000088127.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3000088127.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Cnstr


In [339]:
# 8 to 7

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil', 'Durbl'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/4280011220.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4280011220.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4280011220.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Durbl


/tmp/ipykernel_1917/4280011220.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4280011220.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/4280011220.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [340]:
# 7 to 6

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/2239831781.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2239831781.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Oil


/tmp/ipykernel_1917/2239831781.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2239831781.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/2239831781.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [341]:
# 6 to 5

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/807753741.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/807753741.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Cars


/tmp/ipykernel_1917/807753741.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/807753741.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/807753741.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

In [342]:
# 5 to 4

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other', 'Steel'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/1453898363.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1453898363.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


/tmp/ipykernel_1917/1453898363.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1453898363.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


The HML predictor with the lowest SSE is: Steel


/tmp/ipykernel_1917/1453898363.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [343]:
# 4 to 3

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Machn', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Machn


/tmp/ipykernel_1917/3634139292.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3634139292.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3634139292.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [344]:
# 3 to 2

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Finan', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

/tmp/ipykernel_1917/3039220418.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3039220418.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/3039220418.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML predictor with the lowest SSE is: Finan


In [345]:
# 2 to 1

#store sse
predict_hml_df = pd.DataFrame()

#HML

hml_pred = pd.Series(['Trans', 'Other'])

for pred in hml_pred:
    #remove one variable
    col = hml_pred[~hml_pred.isin([pred])]
    explanatory = seventeen_grouped[col]

    predict_hml = pd.Series()

    for i in range(0,len(group_labels), 4):
        train1 = explanatory.get_group(group_labels[i]).values
        train2 = explanatory.get_group(group_labels[i+1]).values
        train3 = explanatory.get_group(group_labels[i+2]).values
        train4 = explanatory.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        predict = pd.Series(linear.predict(train).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using in-sample SSE
sse_hml = ((predict_hml_df - hml['HML'].values.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

The HML predictor with the lowest SSE is: Other


/tmp/ipykernel_1917/1829747955.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1917/1829747955.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])


In [95]:
# best models

best_model_hml = []
best_model_mom = []

# 1 var
best_model_hml.append('Trans')
best_model_mom.append('Trans')

# 2 var
best_model_hml.append(['Trans', 'Other'])

# 3 var
best_model_hml.append(['Trans', 'Other', 'Finan'])

# 4 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other'])

# 5 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel'])

# 6 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars'])

# 7 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil'])

# 8 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil', 'Durbl'])

# 9 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil', 'Durbl', 'Cnstr'])

# 10 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil', 'Rtail', 'Durbl', 'Cnstr'])

# 11 var
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr'])

# 12 var (diff)
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Oil', 'Rtail', 'Durbl', 
                      'Cnstr', 'Cnsum'])

# 13 var (same as forward)
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum'])

# 14 var (same as forward)
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum', 'Clths'])

# 15 var (same as forward)
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr'])

# 16 var (same as forward)
best_model_hml.append(['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail',
                       'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines'])

In [96]:
#choose best model using forward stepwise selection and out of sample SSE, AIC, BIC

#store predicted value
predict_hml_df = pd.DataFrame()

#HML
for pred in best_model_hml:
    #choose one variable
    explanatory = seventeen_grouped[pred]
    predict_hml = pd.Series()
    # since it's 5-year rolling scheme
    for i in range(0,len(group_labels)-4):
        if isinstance(pred, list):
            train1 = explanatory.get_group(group_labels[i]).values
            train2 = explanatory.get_group(group_labels[i+1]).values
            train3 = explanatory.get_group(group_labels[i+2]).values
            train4 = explanatory.get_group(group_labels[i+3]).values
            test = explanatory.get_group(group_labels[i+4]).values
        else:
            train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
            train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
            train3 = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
            train4 = explanatory.get_group(group_labels[i+3]).values.reshape(-1, 1)
            test = explanatory.get_group(group_labels[i+4]).values.reshape(-1, 1)
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)

        #estimate HML using 20 year data
        linear.fit(train,y)
        #predict next 5 year HML
        predict = pd.Series(linear.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])
    
    predict_hml_df[f'{pred}'] = predict_hml

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_5yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

k_values = list(range(1, 18))

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_values)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_values)]

/tmp/ipykernel_1937/1571729941.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/1571729941.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_1937/1571729941.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [348]:
# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
print(f"The HML predictor with the lowest SSE is: {best_hml}")

# Choose optimal model (minimum AIC)
min_aic = aic_hml.index(min(aic_hml))
best_hml = best_model_hml[min_aic]
print(f"The HML predictor with the lowest AIC is: {best_hml}")

# Choose optimal model (minimum BIC)
min_bic = bic_hml.index(min(bic_hml))
best_hml = best_model_hml[min_bic]
print(f"The HML predictor with the lowest BIC is: {best_hml}")

The HML predictor with the lowest SSE is: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines']
The HML predictor with the lowest AIC is: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines']
The HML predictor with the lowest BIC is: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Oil']


### Ridge

In [47]:
# ridge and lasso (tune alpha)

### ridge

alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-4):
        train1 = seventeen_grouped.get_group(group_labels[i]).values
        train2 = seventeen_grouped.get_group(group_labels[i+1]).values
        train3 = seventeen_grouped.get_group(group_labels[i+2]).values
        train4 = seventeen_grouped.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        test = seventeen_grouped.get_group(group_labels[i+4]).values

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)
        
        #estimate HML using 20 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, y)
        
        #predict next 5 year HML
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)
    
        #estimate MOM using 20 year data
        ridge_model = Ridge(alpha=a, fit_intercept=False)
        ridge_model.fit(train, y)
    
        #predict next 5 year MOM
        predict = pd.Series(ridge_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_ridge_a = float(best_hml)
print(f"The HML alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_ridge_a = float(best_mom)
print(f"The MOM alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_5550/1748544792.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/1748544792.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/1748544792.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

The HML alpha with the lowest SSE is: 0.01
The MOM alpha with the lowest SSE is: 10000.0


/tmp/ipykernel_5550/1748544792.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/1748544792.py:51: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/1748544792.py:37: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

### Lasso

In [78]:
alpha_values = np.logspace(-2, 4, 13)

#store predicted value
predict_hml_df = pd.DataFrame()
predict_mom_df = pd.DataFrame()

for a in alpha_values:

    predict_hml = pd.Series()
    predict_mom = pd.Series()

    for i in range(0,len(group_labels)-4):
        train1 = seventeen_grouped.get_group(group_labels[i]).values
        train2 = seventeen_grouped.get_group(group_labels[i+1]).values
        train3 = seventeen_grouped.get_group(group_labels[i+2]).values
        train4 = seventeen_grouped.get_group(group_labels[i+3]).values
        train = np.concatenate((train1, train2, train3, train4), axis=0)

        test = seventeen_grouped.get_group(group_labels[i+4]).values

        y1 = hml_grouped.get_group(group_labels[i]).values
        y2 = hml_grouped.get_group(group_labels[i+1]).values
        y3 = hml_grouped.get_group(group_labels[i+2]).values
        y4 = hml_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)
        
        #estimate HML using 20 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, y)
        
        #predict next 5 year HML
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_hml = pd.concat([predict_hml, predict])

        y1 = mom_grouped.get_group(group_labels[i]).values
        y2 = mom_grouped.get_group(group_labels[i+1]).values
        y3 = mom_grouped.get_group(group_labels[i+2]).values
        y4 = mom_grouped.get_group(group_labels[i+3]).values
        y = np.concatenate((y1, y2, y3, y4), axis=0)
    
        #estimate MOM using 20 year data
        lasso_model = Lasso(alpha=a, fit_intercept=False)
        lasso_model.fit(train, y)
    
        #predict next 5 year MOM
        predict = pd.Series(lasso_model.predict(test).flatten())
        predict_mom = pd.concat([predict_mom, predict])
    
    predict_hml_df[f'{a}'] = predict_hml
    predict_mom_df[f'{a}'] = predict_mom

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df - hml_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_mom = ((predict_mom_df - mom_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)

# Choose optimal model (minimum SSE)
best_hml = sse_hml.idxmin()
hml_lasso_a = float(best_hml)
print(f"The HML alpha with the lowest SSE is: {best_hml}")
best_mom = sse_mom.idxmin()
mom_lasso_a20 = float(best_mom)
print(f"The MOM alpha with the lowest SSE is: {best_mom}")

/tmp/ipykernel_5550/468384749.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/468384749.py:47: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/468384749.py:33: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operatio

The HML alpha with the lowest SSE is: 0.01
The MOM alpha with the lowest SSE is: 1000.0


/tmp/ipykernel_5550/468384749.py:47: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])


### Random Forest

In [101]:
#random forest regression

test_fold = [-1] * 240 + [0] * 60
cv_splitter = PredefinedSplit(test_fold)

#set seed to 42
rf_model = RandomForestRegressor(random_state=42)

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 200],  # Number of trees
    'max_depth': [None, 10, 20, 30],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum samples required to be at a leaf node
    'max_features': ['sqrt'] #slide 7 said use sqrt p predictors where p is the total number of predictors
}

# Time series cross-validator
# tscv = TimeSeriesSplit(n_splits=5) #this way the cross validation only happens in chronological order

#create model
rf_hml = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=cv_splitter, 
                        n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)
rf_mom = RandomizedSearchCV(estimator=rf_model, param_distributions=param_grid, n_iter=100, cv=cv_splitter, 
                        n_jobs=-1, scoring='neg_mean_squared_error', random_state=42)

# tune hyperparameters

param_hml = []
param_mom = []

for i in range(0,len(group_labels)-5): #
    x_train1 = seventeen_grouped.get_group(group_labels[i]).values
    x_train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    x_train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    x_train4 = seventeen_grouped.get_group(group_labels[i+3]).values

    hml_train1 = hml_grouped.get_group(group_labels[i]).values
    hml_train2 = hml_grouped.get_group(group_labels[i+1]).values
    hml_train3 = hml_grouped.get_group(group_labels[i+2]).values
    hml_train4 = hml_grouped.get_group(group_labels[i+3]).values
    
    mom_train1 = mom_grouped.get_group(group_labels[i]).values
    mom_train2 = mom_grouped.get_group(group_labels[i+1]).values
    mom_train3 = mom_grouped.get_group(group_labels[i+2]).values
    mom_train4 = mom_grouped.get_group(group_labels[i+3]).values

    x_test = seventeen_grouped.get_group(group_labels[i+4]).values
    hml_test = hml_grouped.get_group(group_labels[i+4]).values
    mom_test = mom_grouped.get_group(group_labels[i+4]).values

    x = np.concatenate((x_train1, x_train2, x_train3, x_train4, x_test), axis=0)
    h = np.concatenate((hml_train1, hml_train2, hml_train3, hml_train4, hml_test), axis=0)
    m = np.concatenate((mom_train1, mom_train2, mom_train3, mom_train4, mom_test), axis=0)

    rf_hml.fit(x, h)
    param_hml.append(rf_hml.best_params_)

    rf_mom.fit(x, m)
    param_mom.append(rf_mom.best_params_)

/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/home/codespace/.local/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

In [102]:
print("For HML")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_hml]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_hml]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_hml]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_hml]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

# For HML
# The most commonly appeared value for 'n_estimators' is 50 with 8 occurrences.
# The most commonly appeared value for 'min_samples_split' is 2 with 9 occurrences.
# The most commonly appeared value for 'min_samples_leaf' is 1 with 8 occurrences.
# The most commonly appeared value for 'max_depth' is 20 with 6 occurrences.

For HML
The most commonly appeared value for 'n_estimators' is 50 with 8 occurrences.
The most commonly appeared value for 'min_samples_split' is 2 with 9 occurrences.
The most commonly appeared value for 'min_samples_leaf' is 1 with 8 occurrences.
The most commonly appeared value for 'max_depth' is 20 with 6 occurrences.


In [103]:
print("For MOM")
# Extract the 'n_estimators' values
n_estimators = [p['n_estimators'] for p in param_mom]

# Count the frequency of each value
n_estimators_counts = Counter(n_estimators)

# Get the most common value and its count
most_common_value, count = n_estimators_counts.most_common(1)[0]

print(f"The most commonly appeared value for 'n_estimators' is {most_common_value} with {count} occurrences.")

min_samples_split = [p['min_samples_split'] for p in param_mom]
min_samples_split_counts = Counter(min_samples_split)
most_common_value, count = min_samples_split_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_split' is {most_common_value} with {count} occurrences.")

min_samples_leaf = [p['min_samples_leaf'] for p in param_mom]
min_samples_leaf_counts = Counter(min_samples_leaf)
most_common_value, count = min_samples_leaf_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'min_samples_leaf' is {most_common_value} with {count} occurrences.")

max_depth = [p['max_depth'] for p in param_mom]
max_depth_counts = Counter(max_depth)
most_common_value, count = max_depth_counts.most_common(1)[0]
print(f"The most commonly appeared value for 'max_depth' is {most_common_value} with {count} occurrences.")

# For MOM
# The most commonly appeared value for 'n_estimators' is 50 with 5 occurrences.
# The most commonly appeared value for 'min_samples_split' is 10 with 9 occurrences.
# The most commonly appeared value for 'min_samples_leaf' is 4 with 9 occurrences.
# The most commonly appeared value for 'max_depth' is 10 with 6 occurrences.

For MOM
The most commonly appeared value for 'n_estimators' is 50 with 5 occurrences.
The most commonly appeared value for 'min_samples_split' is 10 with 9 occurrences.
The most commonly appeared value for 'min_samples_leaf' is 4 with 9 occurrences.
The most commonly appeared value for 'max_depth' is 10 with 6 occurrences.


In [88]:
# overal model selection

#store predicted value
predict_hml_df20 = pd.DataFrame()
predict_mom_df20 = pd.DataFrame()

#linear
# HML: ['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 'Durbl', 'Cnstr', 
# 'Cnsum', 'Clths', 'FabPr', 'Mines']
predict_hml = pd.Series()
explanatory = seventeen_grouped[['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 
                                 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines']]
for i in range(0,len(group_labels)-4):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train3 = explanatory.get_group(group_labels[i+2]).values
    train4 = explanatory.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = explanatory.get_group(group_labels[i+4]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y3 = hml_grouped.get_group(group_labels[i+2]).values
    y4 = hml_grouped.get_group(group_labels[i+3]).values
    y = np.concatenate((y1, y2, y3, y4), axis=0)

    #estimate HML using 5 year data
    linear.fit(train,y)
    #predict next 5 year HML
    predict = pd.Series(linear.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])
    
predict_hml_df20['Linear'] = predict_hml

# MOM: 'Trans'
predict_mom = pd.Series()
explanatory = seventeen_grouped['Trans']
for i in range(0,len(group_labels)-4):
    train1 = explanatory.get_group(group_labels[i]).values.reshape(-1, 1)
    train2 = explanatory.get_group(group_labels[i+1]).values.reshape(-1, 1)
    train3 = explanatory.get_group(group_labels[i+2]).values.reshape(-1, 1)
    train4 = explanatory.get_group(group_labels[i+3]).values.reshape(-1, 1)
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = explanatory.get_group(group_labels[i+4]).values.reshape(-1, 1)
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y3 = mom_grouped.get_group(group_labels[i+2]).values
    y4 = mom_grouped.get_group(group_labels[i+3]).values
    y = np.concatenate((y1, y2, y3, y4), axis=0)

    #estimate MOM using 20 year data
    linear.fit(train,y)
    #predict next 5 year MOM
    predict = pd.Series(linear.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])
    
predict_mom_df20['Linear'] = predict_mom

#ridge
# HML alpha: 0.01 (hml_ridge_a)

predict_hml = pd.Series()

for i in range(0,len(group_labels)-4):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+4]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y3 = hml_grouped.get_group(group_labels[i+2]).values
    y4 = hml_grouped.get_group(group_labels[i+3]).values
    y = np.concatenate((y1, y2, y3, y4), axis=0)

    #estimate HML using 20 year data
    ridge_model = Ridge(alpha=hml_ridge_a, fit_intercept=False)
    ridge_model.fit(train, y)
    hml_coef_ridge = ridge_model.coef_
        
    #predict next 5 year HML
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df20['Ridge'] = predict_hml

# MOM alpha: 10000.0 (mom_ridge_a)

predict_mom = pd.Series()

for i in range(0,len(group_labels)-4):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+4]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y3 = mom_grouped.get_group(group_labels[i+2]).values
    y4 = mom_grouped.get_group(group_labels[i+3]).values
    y = np.concatenate((y1, y2, y3, y4), axis=0)

    #estimate MOM using 5 year data
    ridge_model = Ridge(alpha=mom_ridge_a, fit_intercept=False)
    ridge_model.fit(train, y)
    mom_coef_ridge = ridge_model.coef_
        
    #predict next 5 year MOM
    predict = pd.Series(ridge_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df20['Ridge'] = predict_mom

#lasso
# HML alpha: 0.01 (hml_lasso_a)
predict_hml = pd.Series()

for i in range(0,len(group_labels)-4):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+4]).values
        
    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y3 = hml_grouped.get_group(group_labels[i+2]).values
    y4 = hml_grouped.get_group(group_labels[i+3]).values
    y = np.concatenate((y1, y2, y3, y4), axis=0)

    #estimate HML using 5 year data
    lasso_model = Lasso(alpha=hml_lasso_a, fit_intercept=False)
    lasso_model.fit(train, y)
        
    #predict next 5 year HML
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_hml = pd.concat([predict_hml, predict])

predict_hml_df20['Lasso'] = predict_hml

# MOM alpha: 1000.0 (mom_lasso_a)

predict_mom = pd.Series()

for i in range(0,len(group_labels)-4):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+4]).values
        
    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y3 = mom_grouped.get_group(group_labels[i+2]).values
    y4 = mom_grouped.get_group(group_labels[i+3]).values
    y = np.concatenate((y1, y2, y3, y4), axis=0)

    #estimate MOM using 20 year data
    lasso_model = Lasso(alpha=mom_lasso_a20, fit_intercept=False)
    lasso_model.fit(train, y)
        
    #predict next 5 year MOM
    predict = pd.Series(lasso_model.predict(test).flatten())
    predict_mom = pd.concat([predict_mom, predict])

predict_mom_df20['Lasso'] = predict_mom

# random forest

#HML

random_pred_hml = pd.Series()

rf_model_hml = RandomForestRegressor(n_estimators=50, max_depth=20, min_samples_split=2, min_samples_leaf=1,
                                 random_state=42)

for i in range(0,len(group_labels)-4): 
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    x_train = np.concatenate((train1, train2, train3, train4), axis=0)

    y1 = hml_grouped.get_group(group_labels[i]).values
    y2 = hml_grouped.get_group(group_labels[i+1]).values
    y3 = hml_grouped.get_group(group_labels[i+2]).values
    y4 = hml_grouped.get_group(group_labels[i+3]).values
    hml_train = np.concatenate((y1, y2, y3, y4), axis=0)

    x_test = seventeen_grouped.get_group(group_labels[i+4]).values

    rf_model_hml.fit(x_train, hml_train)

    #predict next 5 year HML
    predict = pd.Series(rf_model_hml.predict(x_test).flatten())
    random_pred_hml = pd.concat([random_pred_hml, predict])

#MOM

random_pred_mom = pd.Series()

rf_model_mom = RandomForestRegressor(n_estimators=50, max_depth=10, min_samples_split=10, min_samples_leaf=4,
                                 random_state=42)

for i in range(0,len(group_labels)-4): 
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    x_train = np.concatenate((train1, train2, train3, train4), axis=0)

    y1 = mom_grouped.get_group(group_labels[i]).values
    y2 = mom_grouped.get_group(group_labels[i+1]).values
    y3 = mom_grouped.get_group(group_labels[i+2]).values
    y4 = mom_grouped.get_group(group_labels[i+3]).values
    mom_train = np.concatenate((y1, y2, y3, y4), axis=0)

    x_test = seventeen_grouped.get_group(group_labels[i+4]).values

    rf_model_mom.fit(x_train, mom_train)

    #predict next 5 year MOM
    predict = pd.Series(rf_model_mom.predict(x_test).flatten())
    random_pred_mom = pd.concat([random_pred_mom, predict])

# pick best 2

#compare using out-of-sample SSE
sse_hml = ((predict_hml_df20 - hml_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_rf_hml = ((random_pred_hml - hml_20yr_actual.flatten()) ** 2).sum()
sse_mom = ((predict_mom_df20 - mom_20yr_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_rf_mom = ((random_pred_mom - mom_20yr_actual.flatten()) ** 2).sum()

# Compute SST
hml_mean = hml_20yr_actual.mean()
sst_hml20 = ((hml_20yr_actual - hml_mean) ** 2).sum()

mom_mean = mom_20yr_actual.mean()
sst_mom20 = ((mom_20yr_actual - mom_mean) ** 2).sum()

# calculate R^2
r2_hml = 1 - (sse_hml / sst_hml)
r2_rf_hml = 1 - (sse_rf_hml / sst_hml)
r2_mom = 1 - (sse_mom / sst_mom)
r2_rf_mom = 1 - (sse_rf_mom / sst_mom)

/tmp/ipykernel_5550/1293650332.py:32: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_hml = pd.concat([predict_hml, predict])
/tmp/ipykernel_5550/1293650332.py:58: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  predict_mom = pd.concat([predict_mom, predict])
/tmp/ipykernel_5550/1293650332.py:89: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat opera

In [50]:
# count num of non zero coef
k_ridge_hml = np.count_nonzero(hml_coef_ridge)
k_ridge_mom = np.count_nonzero(mom_coef_ridge)

k_lasso_hml = np.count_nonzero(hml_coef_lasso)
k_lasso_mom = np.count_nonzero(mom_coef_lasso)

k_l_hml = 16
k_l_mom = 1

k_hml = [k_l_hml, k_ridge_hml, k_lasso_hml]

k_mom = [k_l_mom, k_ridge_mom, k_lasso_mom]

# Calculate AIC and BIC for HML
aic_hml = [np.log(sse / n_hml) + 2 * k / n_hml for sse, k in zip(sse_hml, k_hml)]
bic_hml = [np.log(sse / n_hml) + k * np.log(n_hml) / n_hml for sse, k in zip(sse_hml, k_hml)]
aic_hml = pd.Series(aic_hml)
bic_hml = pd.Series(bic_hml)
aic_hml.index = sse_hml.index
bic_hml.index = sse_hml.index

# Calculate AIC and BIC for MOM
aic_mom = [np.log(sse / n_mom) + 2 * k / n_mom for sse, k in zip(sse_mom, k_mom)]
bic_mom = [np.log(sse / n_mom) + k * np.log(n_mom) / n_mom for sse, k in zip(sse_mom, k_mom)]
aic_mom = pd.Series(aic_mom)
bic_mom = pd.Series(bic_mom)
aic_mom.index = sse_mom.index
bic_mom.index = sse_mom.index

In [51]:
#sort in ascending order
sse_hml = pd.concat([sse_hml, pd.Series(sse_rf_hml)]).sort_values()
sse_mom = pd.concat([sse_mom, pd.Series(sse_rf_mom)]).sort_values()
aic_hml = aic_hml.sort_values()
aic_mom = aic_mom.sort_values()
bic_hml = bic_hml.sort_values()
bic_mom = bic_mom.sort_values()
r2_hml = pd.concat([r2_hml, pd.Series(r2_rf_hml)]).sort_values(ascending=False)
r2_mom = pd.concat([r2_mom, pd.Series(r2_rf_mom)]).sort_values(ascending=False)

In [52]:
print(r2_hml)
print(bic_hml)

# based on this the best 2 for 20 year HML are Linear and Lasso

Linear    0.467272
Lasso     0.464100
Ridge     0.464013
0         0.191185
0         0.191185
dtype: float64
Lasso     1.551466
Linear    1.567498
Ridge     1.580919
dtype: float64


In [53]:
print(r2_mom)
print(bic_mom)

# based on this the best two are lasso and RF

Lasso    -0.006356
0        -0.021782
0        -0.021782
Ridge    -0.139216
Linear   -0.194839
dtype: float64
Lasso     5.099511
Linear    5.263864
Ridge     5.333358
dtype: float64


In [205]:
# Choose optimal model (minimum SSE)
# best_hml = sse_hml.idxmin()
# print(f"The HML predictor with the lowest SSE is: {best_hml}")
# second = sse_hml.index[1]
# print(f"The HML predictor with the second lowest SSE is: {second}")
# Choose optimal model (max R2)
# best_hml = r2_hml.idxmax()
# print(f"The HML predictor with the highest R2 is: {best_hml}")
# second = r2_hml.index[2]
# print(f"The HML predictor with the second highest R2 is: {second}")
# Choose optimal model (minimum AIC)
# best_hml = aic_hml.idxmin()
# print(f"The HML predictor with the lowest AIC is: {best_hml}")
# second = aic_hml.index[1]
# print(f"The HML predictor with the second lowest AIC is: {second}")
# Choose optimal model (minimum BIC)
# best_hml = bic_hml.idxmin()
# print(f"The HML predictor with the lowest BIC is: {best_hml}")
# second = bic_hml.index[1]
# print(f"The HML predictor with the second lowest BIC is: {second}")

# Choose optimal model (minimum SSE)
# best_mom = sse_mom.idxmin()
# print(f"The MOM predictor with the lowest SSE is: {best_mom}")
# second = sse_mom.index[1]
# print(f"The MOM predictor with the second lowest SSE is: {second}")
# Choose optimal model (max R2)
# best_mom = r2_mom.idxmax()
# print(f"The MOM predictor with the highest R2 is: {best_mom}")
# second = r2_mom.index[2]
# print(f"The MOM predictor with the second highest R2 is: {second}")
# Choose optimal model (minimum AIC)
# best_mom = aic_mom.idxmin()
# print(f"The MOM predictor with the lowest AIC is: {best_mom}")
# second = aic_mom.index[1]
# print(f"The MOM predictor with the second lowest AIC is: {second}")
# Choose optimal model (minimum BIC)
# best_mom = bic_mom.idxmin()
# print(f"The MOM predictor with the lowest BIC is: {best_mom}")
# second = bic_mom.index[1]
# print(f"The MOM predictor with the second lowest BIC is: {second}")

The HML predictor with the lowest SSE is: Linear
The HML predictor with the second lowest SSE is: Lasso
The HML predictor with the highest R2 is: Linear
The HML predictor with the second highest R2 is: Lasso
The HML predictor with the lowest AIC is: Linear
The HML predictor with the second lowest AIC is: Lasso
The HML predictor with the lowest BIC is: Linear
The HML predictor with the second lowest BIC is: Lasso
The MOM predictor with the lowest SSE is: Lasso
The MOM predictor with the second lowest SSE is: 0
The MOM predictor with the highest R2 is: Lasso
The MOM predictor with the second highest R2 is: 0
The MOM predictor with the lowest AIC is: Lasso
The MOM predictor with the second lowest AIC is: Ridge
The MOM predictor with the lowest BIC is: Lasso
The MOM predictor with the second lowest BIC is: Linear


In [54]:
#get values of the two best methods so I don't have to calculate them again :P

# twenty_sse_hml_linear = sse_hml.loc['Linear']
# twenty_sse_hml_lasso = sse_hml.loc['Lasso']

# twenty_aic_hml_linear = aic_hml.loc['Linear']
# twenty_aic_hml_lasso = aic_hml.loc['Lasso']

twenty_bic_hml_linear = bic_hml.loc['Linear']
twenty_bic_hml_lasso = bic_hml.loc['Lasso']

twenty_r2_hml_linear = r2_hml.loc['Linear']
twenty_r2_hml_lasso = r2_hml.loc['Lasso']

# twenty_sse_mom_random_forest = sse_rf_mom
# twenty_sse_mom_lasso = sse_mom.loc['Lasso']

# twenty_aic_mom_lasso = aic_mom.loc['Lasso']

twenty_bic_mom_lasso = bic_mom.loc['Lasso']

twenty_r2_mom_random_forest = r2_rf_mom
twenty_r2_mom_lasso = r2_mom.loc['Lasso']

In [63]:
# compare across three training methods 1a, 1b, 1c to select two best overall

bests_hml_bic = {'five_bic_hml_ridge': five_bic_hml_ridge, 'five_bic_hml_linear': five_bic_hml_linear,
                 'ten_bic_hml_lasso': ten_bic_hml_lasso, 'ten_bic_hml_linear': ten_bic_hml_linear,
                 'twenty_bic_hml_lasso': twenty_bic_hml_lasso, 'twenty_bic_hml_linear': twenty_bic_hml_linear}

bests_hml_r2 = {'five_r2_hml_ridge': five_r2_hml_ridge, 'five_r2_hml_linear': five_r2_hml_linear,
                'ten_r2_hml_lasso': ten_r2_hml_lasso, 'ten_r2_hml_linear': ten_r2_hml_linear,
                'twenty_r2_hml_lasso': twenty_r2_hml_lasso, 'twenty_r2_hml_linear': twenty_r2_hml_linear}

bests_hml_bic = dict(sorted(bests_hml_bic.items(), key=lambda item: item[1]))
bests_hml_r2 = dict(sorted(bests_hml_r2.items(), key=lambda item: item[1], reverse=True))

bests_mom_bic = {'five_bic_mom_linear': five_bic_mom_linear, 'five_bic_mom_ridge': five_bic_mom_ridge,
                 'ten_bic_mom_lasso': ten_bic_mom_lasso,
                 'twenty_bic_mom_lasso': twenty_bic_mom_lasso}

bests_mom_r2 = {'five_r2_mom_linear': five_r2_mom_linear, 'five_r2_mom_ridge': five_r2_mom_ridge,
                'ten_r2_mom_lasso': ten_r2_mom_lasso, 'ten_r2_mom_random_forest': ten_r2_mom_random_forest,
                'twenty_r2_mom_lasso': twenty_r2_mom_lasso, 'twenty_r2_mom_random_forest': twenty_r2_mom_random_forest}

bests_mom_bic = dict(sorted(bests_mom_bic.items(), key=lambda item: item[1]))
bests_mom_r2 = dict(sorted(bests_mom_r2.items(), key=lambda item: item[1], reverse=True))

In [66]:
bests_hml_bic
#optimal for HML based on BIC are ten-year training linear model and ten-year training Lasso model

{'ten_bic_hml_linear': np.float64(1.3327225587557945),
 'ten_bic_hml_lasso': np.float64(1.4309885797228588),
 'twenty_bic_hml_lasso': np.float64(1.5514662284370453),
 'twenty_bic_hml_linear': np.float64(1.5674982283297827),
 'five_bic_hml_linear': np.float64(1.5745419039383115),
 'five_bic_hml_ridge': np.float64(1.7180219291365941)}

In [67]:
bests_hml_r2

# overall ranking
# 'ten_bic_hml_linear': 1+1
# 'ten_bic_hml_lasso': 2+4
# 'twenty_bic_hml_lasso': 3+3 
# 'twenty_bic_hml_linear': 4+2
# 'five_bic_hml_linear': 5+5
# 'five_bic_hml_ridge': 6+6

# overall, optimal for HML based on R2 are ten-year training linear model and twenty-year training linear model 
# (twenty year linear model selected because has best R2 out of the three in second place)

{'ten_r2_hml_linear': np.float64(0.48629519818162403),
 'twenty_r2_hml_linear': np.float64(0.46727190254890605),
 'twenty_r2_hml_lasso': np.float64(0.46410004221193724),
 'ten_r2_hml_lasso': np.float64(0.45885428434574627),
 'five_r2_hml_linear': np.float64(0.4185032327968585),
 'five_r2_hml_ridge': np.float64(0.38525221451858194)}

In [68]:
bests_mom_bic

{'ten_bic_mom_lasso': np.float64(4.977201883273039),
 'twenty_bic_mom_lasso': np.float64(5.099511243023343),
 'five_bic_mom_linear': np.float64(5.150708370199653),
 'five_bic_mom_ridge': np.float64(5.266438655819387)}

In [69]:
bests_mom_r2

# overall ranking: 
# 'ten_r2_mom_lasso': 1+1
# 'twenty_r2_mom_lasso': 2+2
# 'twenty_r2_mom_random_forest': 3
# 'five_r2_mom_linear': 4+3
# 'five_r2_mom_ridge': 5+4
# 'ten_r2_mom_random_forest': 6

# overall optimal for MOM are ten-year training Lasso model and twenty-year training lasso model

{'ten_r2_mom_lasso': np.float64(-0.0063560159402964445),
 'twenty_r2_mom_lasso': np.float64(-0.0063560159402964445),
 'twenty_r2_mom_random_forest': np.float64(-0.021782132927745446),
 'five_r2_mom_linear': np.float64(-0.05922025488635341),
 'five_r2_mom_ridge': np.float64(-0.06547596601118633),
 'ten_r2_mom_random_forest': np.float64(-0.07525979645148317)}

In [215]:
bests_mom_sse
#optimal for MOM based on SSE are ten-year training Lasso model and twenty-year training lasso model

#overall, optimal for MOM are ten-year training Lasso model and five-year training Linear model

{'ten_sse_mom_lasso': np.float64(150895.3941),
 'twenty_sse_mom_lasso': np.float64(150895.3941),
 'twenty_sse_mom_random_forest': np.float64(153208.42245714547),
 'five_sse_mom_linear': np.float64(158821.98274577712),
 'five_sse_mom_ridge': np.float64(159759.97882331378),
 'ten_sse_mom_random_forest': np.float64(161226.9894306054)}

# Analysis 2

In [70]:
#HML (10 and 20-year training linear models)

# 20-year Linear

pred_sv_bv = pd.DataFrame()
pred_sg_bg = pd.DataFrame()

v = pd.Series()
g = pd.Series()

explanatory = seventeen_grouped[['Trans', 'Machn', 'Finan', 'Other', 'Steel', 'Cars', 'Utils', 'Chems', 'Oil', 'Rtail', 
                                 'Durbl', 'Cnstr', 'Cnsum', 'Clths', 'FabPr', 'Mines']]
for i in range(0,len(group_labels)-4): #range start from 2 so the predictions will be the same as twenty-year training (for MOM)
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train3 = explanatory.get_group(group_labels[i+2]).values
    train4 = explanatory.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = explanatory.get_group(group_labels[i+4]).values
        
    y1 = size_value_grouped.get_group(group_labels[i])
    sv_bv1 = y1['BIG HiBM'] + y1['SMALL HiBM']
    sg_bg1 = y1['BIG LoBM'] + y1['SMALL LoBM']
    y2 = size_value_grouped.get_group(group_labels[i+1])
    sv_bv2 = y2['BIG HiBM'] + y2['SMALL HiBM']
    sg_bg2 = y2['BIG LoBM'] + y2['SMALL LoBM']
    y3 = size_value_grouped.get_group(group_labels[i+2])
    sv_bv3 = y3['BIG HiBM'] + y3['SMALL HiBM']
    sg_bg3 = y3['BIG LoBM'] + y3['SMALL LoBM']
    y4 = size_value_grouped.get_group(group_labels[i+3])
    sv_bv4 = y4['BIG HiBM'] + y4['SMALL HiBM']
    sg_bg4 = y4['BIG LoBM'] + y4['SMALL LoBM']
    sv_bv = np.concatenate((sv_bv1.values, sv_bv2.values, sv_bv3.values, sv_bv4.values), axis=0)
    sg_bg = np.concatenate((sg_bg1.values, sg_bg2.values, sg_bg3.values, sg_bg4.values), axis=0)

    linear.fit(train,sv_bv)
    #predict next 5 year SV+BV
    predict = pd.Series(linear.predict(test).flatten())
    v = pd.concat([v, predict])

    linear.fit(train,sg_bg)
    #predict next 5 year SG+BG
    predict = pd.Series(linear.predict(test).flatten())
    g = pd.concat([g, predict])

pred_sv_bv['Linear 20Yr'] = v
pred_sg_bg['Linear 20Yr'] = g

v = pd.Series()
g = pd.Series()

# 10-year Linear

explanatory = seventeen_grouped[['Trans', 'Machn', 'Steel', 'Finan', 'Other', 'Cars']]
for i in range(2,len(group_labels)-2):
    train1 = explanatory.get_group(group_labels[i]).values
    train2 = explanatory.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = explanatory.get_group(group_labels[i+2]).values
        
    y1 = size_value_grouped.get_group(group_labels[i])
    sv_bv1 = y1['BIG HiBM'] + y1['SMALL HiBM']
    sg_bg1 = y1['BIG LoBM'] + y1['SMALL LoBM']
    y2 = size_value_grouped.get_group(group_labels[i+1])
    sv_bv2 = y2['BIG HiBM'] + y2['SMALL HiBM']
    sg_bg2 = y2['BIG LoBM'] + y2['SMALL LoBM']
    sv_bv = np.concatenate((sv_bv1.values, sv_bv2.values), axis=0)
    sg_bg = np.concatenate((sg_bg1.values, sg_bg2.values), axis=0)

    #estimate SV+BV using 10 year data
    linear.fit(train,sv_bv)
    #predict next 5 year SV+BV
    predict = pd.Series(linear.predict(test).flatten())
    v = pd.concat([v, predict])

    #estimate Sg+BG using 10 year data
    linear.fit(train,sg_bg)
    #predict next 5 year SG+BG
    predict = pd.Series(linear.predict(test).flatten())
    g = pd.concat([g, predict])

pred_sv_bv['Linear 10Yr'] = v
pred_sg_bg['Linear 10Yr'] = g

/tmp/ipykernel_5550/4019519366.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  v = pd.concat([v, predict])
/tmp/ipykernel_5550/4019519366.py:45: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  g = pd.concat([g, predict])
/tmp/ipykernel_5550/4019519366.py:76: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  v = pd.concat([v, predict])
/tmp

In [71]:
value_to_exclude1 = size_value_grouped.get_group(group_labels[0])
value_to_exclude2 = size_value_grouped.get_group(group_labels[1])
value_to_exclude3 = size_value_grouped.get_group(group_labels[2])
value_to_exclude4 = size_value_grouped.get_group(group_labels[3])
#actual response
value_20yr_actual = size_value[~size_value.index.isin(value_to_exclude1.index)]
value_20yr_actual = value_20yr_actual[~value_20yr_actual.index.isin(value_to_exclude2.index)]
value_20yr_actual = value_20yr_actual[~value_20yr_actual.index.isin(value_to_exclude3.index)]
value_20yr_actual = value_20yr_actual[~value_20yr_actual.index.isin(value_to_exclude4.index)]

In [72]:
sv_bv_actual = value_20yr_actual['BIG HiBM'] + value_20yr_actual['SMALL HiBM']
sv_bv_actual = sv_bv_actual.values
sg_bg_actual = value_20yr_actual['BIG LoBM'] + value_20yr_actual['SMALL LoBM']
sg_bg_actual = sg_bg_actual.values

In [73]:
sse_sv_bv_ind = ((pred_sv_bv - sv_bv_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_sg_bg_ind = ((pred_sg_bg - sg_bg_actual.reshape(-1, 1)) ** 2).sum(axis=0)

In [76]:
# Compute SST
sv_bv_mean = sv_bv_actual.mean()
sst_sv_bv = ((sv_bv_actual - sv_bv_mean) ** 2).sum()

sg_bg_mean = sg_bg_actual.mean()
sst_sg_bg = ((sg_bg_actual - sg_bg_mean) ** 2).sum()

# calculate R^2
r2_sv_bv_ind = 1 - (sse_sv_bv_ind / sst_sv_bv)

r2_sg_bg_ind = 1 - (sse_sg_bg_ind / sst_sg_bg)

In [83]:
print(f"R2 of SV+BV using 10-year and 20-year training Linear are")
print(r2_sv_bv_ind)
print(f"R2 of SG+BG using 10-year and 20-year training Linear are:")
print(r2_sg_bg_ind)
print(f"R2 of HML using 20-year training Linear is: {twenty_r2_hml_linear}")
print(f"R2 of HML using 10-year training Linear is: {ten_r2_hml_linear}")

R2 of SV+BV using 10-year and 20-year training Linear are
Linear 20Yr    0.872971
Linear 10Yr    0.859628
dtype: float64
R2 of SG+BG using 10-year and 20-year training Linear are:
Linear 20Yr    0.924218
Linear 10Yr    0.914892
dtype: float64
R2 of HML using 20-year training Linear is: 0.46727190254890605
R2 of HML using 10-year training Linear is: 0.48629519818162403


In [79]:
#mom (10 and 20-year training lasso model)

# 20 year lasso

pred_sh_bh = pd.DataFrame()
pred_sl_bl = pd.DataFrame()

h = pd.Series()
l = pd.Series()

for i in range(0,len(group_labels)-4):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train3 = seventeen_grouped.get_group(group_labels[i+2]).values
    train4 = seventeen_grouped.get_group(group_labels[i+3]).values
    train = np.concatenate((train1, train2, train3, train4), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+4]).values
        
    y1 = size_momentum_grouped.get_group(group_labels[i])
    sh_bh1 = y1['B HiPRIOR'] + y1['SMALL HiPRIOR']
    sl_bl1 = y1['BIG LoPRIO'] + y1['SMALL LoPRIOR']
    y2 = size_momentum_grouped.get_group(group_labels[i+1])
    sh_bh2 = y2['B HiPRIOR'] + y2['SMALL HiPRIOR']
    sl_bl2 = y2['BIG LoPRIO'] + y2['SMALL LoPRIOR']
    y3 = size_momentum_grouped.get_group(group_labels[i+2])
    sh_bh3 = y3['B HiPRIOR'] + y3['SMALL HiPRIOR']
    sl_bl3 = y3['BIG LoPRIO'] + y3['SMALL LoPRIOR']
    y4 = size_momentum_grouped.get_group(group_labels[i+3])
    sh_bh4 = y4['B HiPRIOR'] + y4['SMALL HiPRIOR']
    sl_bl4 = y4['BIG LoPRIO'] + y4['SMALL LoPRIOR']
    sh_bh = np.concatenate((sh_bh1.values, sh_bh2.values, sh_bh3.values, sh_bh4.values), axis=0)
    sl_bl = np.concatenate((sl_bl1.values, sl_bl2.values, sl_bl3.values, sl_bl4.values), axis=0)

    #estimate SH+BH using 5 year data
    lasso_model = Lasso(alpha=mom_lasso_a20, fit_intercept=False)
    lasso_model.fit(train, sh_bh)
    #predict next 5 year SH+BH
    predict = pd.Series(lasso_model.predict(test).flatten())
    h = pd.concat([h, predict])

    #estimate SL+BL using 5 year data
    lasso_model.fit(train, sl_bl)
    #predict next 5 year SL+BL
    predict = pd.Series(lasso_model.predict(test).flatten())
    l = pd.concat([l, predict])

pred_sh_bh['Lasso 20Yr'] = h
pred_sl_bl['Lasso 20Yr'] = l

h = pd.Series()
l = pd.Series()

# 10-year lasso

for i in range(2,len(group_labels)-2):
    train1 = seventeen_grouped.get_group(group_labels[i]).values
    train2 = seventeen_grouped.get_group(group_labels[i+1]).values
    train = np.concatenate((train1, train2), axis=0)

    test = seventeen_grouped.get_group(group_labels[i+2]).values
        
    y1 = size_momentum_grouped.get_group(group_labels[i])
    sh_bh1 = y1['B HiPRIOR'] + y1['SMALL HiPRIOR']
    sl_bl1 = y1['BIG LoPRIO'] + y1['SMALL LoPRIOR']
    y2 = size_momentum_grouped.get_group(group_labels[i+1])
    sh_bh2 = y2['B HiPRIOR'] + y2['SMALL HiPRIOR']
    sl_bl2 = y2['BIG LoPRIO'] + y2['SMALL LoPRIOR']
    sh_bh = np.concatenate((sh_bh1.values, sh_bh2.values), axis=0)
    sl_bl = np.concatenate((sl_bl1.values, sl_bl2.values), axis=0)

    #estimate SH+BH using 10 year data
    lasso_model = Lasso(alpha=mom_lasso_a10, fit_intercept=False)
    lasso_model.fit(train, sh_bh)
        
    #predict next 5 year SH+BH
    predict = pd.Series(lasso_model.predict(test).flatten())
    h = pd.concat([h, predict])

    #estimate SL+BL using 10 year data
    lasso_model = Lasso(alpha=mom_lasso_a10, fit_intercept=False)
    lasso_model.fit(train, sl_bl)
        
    #predict next 5 year SL+BL
    predict = pd.Series(lasso_model.predict(test).flatten())
    l = pd.concat([l, predict])

pred_sh_bh['Lasso 10Yr'] = h
pred_sl_bl['Lasso 10Yr'] = l

/tmp/ipykernel_5550/2317455608.py:40: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  h = pd.concat([h, predict])
/tmp/ipykernel_5550/2317455608.py:46: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  l = pd.concat([l, predict])
/tmp/ipykernel_5550/2317455608.py:78: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  h = pd.concat([h, predict])
/tmp

In [80]:
momentum_to_exclude1 = size_momentum_grouped.get_group(group_labels[0])
momentum_to_exclude2 = size_momentum_grouped.get_group(group_labels[1])
momentum_to_exclude3 = size_momentum_grouped.get_group(group_labels[2])
momentum_to_exclude4 = size_momentum_grouped.get_group(group_labels[3])
#actual response
momentum_20yr_actual = size_momentum[~size_momentum.index.isin(momentum_to_exclude1.index)]
momentum_20yr_actual = momentum_20yr_actual[~momentum_20yr_actual.index.isin(momentum_to_exclude2.index)]
momentum_20yr_actual = momentum_20yr_actual[~momentum_20yr_actual.index.isin(momentum_to_exclude3.index)]
momentum_20yr_actual = momentum_20yr_actual[~momentum_20yr_actual.index.isin(momentum_to_exclude4.index)]

In [81]:
sh_bh_actual = momentum_20yr_actual['B HiPRIOR'] + momentum_20yr_actual['SMALL HiPRIOR']
sh_bh_actual = sh_bh_actual.values
sl_bl_actual = momentum_20yr_actual['BIG LoPRIO'] + momentum_20yr_actual['SMALL LoPRIOR']
sl_bl_actual = sl_bl_actual.values

In [82]:
sse_sl_bl_ind = ((pred_sl_bl - sl_bl_actual.reshape(-1, 1)) ** 2).sum(axis=0)
sse_sh_bh_ind = ((pred_sh_bh - sh_bh_actual.reshape(-1, 1)) ** 2).sum(axis=0)

In [85]:
# Compute SST
sh_bh_mean = sh_bh_actual.mean()
sst_sh_bh = ((sh_bh_actual - sh_bh_mean) ** 2).sum()

sl_bl_mean = sl_bl_actual.mean()
sst_sl_bl = ((sl_bl_actual - sl_bl_mean) ** 2).sum()

# calculate R^2
r2_sh_bh_ind = 1 - (sse_sh_bh_ind / sst_sh_bh)

r2_sl_bl_ind = 1 - (sse_sl_bl_ind / sst_sl_bl)

In [86]:
print(f"R2 of SH+BH using 10-year and 20-year trainig LASSO are:")
print(r2_sh_bh_ind)
print(f"R2 of SL+BL using 10-year and 20-year trainig LASSO are:")
print(r2_sl_bl_ind)
print(f"R2 of MOM using 10-year training LASSO is: {ten_r2_mom_lasso}")
print(f"R2 of MOM using 20-year training LASSO is: {twenty_r2_mom_lasso}")

R2 of SH+BH using 10-year and 20-year trainig LASSO are:
Lasso 20Yr   -0.076024
Lasso 10Yr   -0.076024
dtype: float64
R2 of SL+BL using 10-year and 20-year trainig LASSO are:
Lasso 20Yr   -0.01441
Lasso 10Yr   -0.01441
dtype: float64
R2 of MOM using 10-year training LASSO is: -0.0063560159402964445
R2 of MOM using 20-year training LASSO is: -0.0063560159402964445


# Analysis 3

In [87]:
pred_hml_avg = (predict_hml_df10['Linear'][120:] + predict_hml_df20['Linear']) / 2
pred_sg_bg['Mean'] = pred_sg_bg.mean(axis=1)
pred_sv_bv['Mean'] = pred_sv_bv.mean(axis=1)

pred_mom_avg = (predict_mom_df10['Lasso'][120:] + predict_mom_df20['Lasso']) / 2
pred_sl_bl['Mean'] = pred_sl_bl.mean(axis=1)
pred_sh_bh['Mean'] = pred_sh_bh.mean(axis=1) 

In [89]:
sse_hml_comb = ((pred_hml_avg - hml_20yr_actual.flatten()) ** 2).sum()
sse_sg_bg_comb = ((pred_sg_bg['Mean'] - sg_bg_actual) ** 2).sum()
sse_sv_bv_comb = ((pred_sv_bv['Mean'] - sv_bv_actual) ** 2).sum()
sse_mom_comb = ((pred_mom_avg - mom_20yr_actual.flatten()) ** 2).sum()
sse_sh_bh_comb = ((pred_sh_bh['Mean'] - sh_bh_actual) ** 2).sum()
sse_sl_bl_comb = ((pred_sl_bl['Mean'] - sl_bl_actual) ** 2).sum()

In [90]:
r2_hml_comb = 1 - (sse_hml_comb / sst_hml20)
r2_sg_bg_comb = 1 - (sse_sg_bg_comb / sst_sg_bg)
r2_sv_bv_comb = 1 - (sse_sv_bv_comb / sst_sv_bv)
r2_mom_comb = 1 - (sse_mom_comb / sst_mom20)
sse_sh_bh_comb = 1 - (sse_sh_bh_comb / sst_sh_bh)
sse_sl_bl_comb = 1 - (sse_sl_bl_comb / sst_sl_bl)

In [92]:
print(f"R2 of HML prediction using 10-year Linear only is: {ten_r2_hml_linear}")
print(f"R2 of HML prediction using 20-year Linear only is: {twenty_r2_hml_linear}")
print(f"R2 of HML prediction using a combination of 10 and 20-year Linear is: {r2_hml_comb}")

R2 of HML prediction using 10-year Linear only is: 0.48629519818162403
R2 of HML prediction using 20-year Linear only is: 0.46727190254890605
R2 of HML prediction using a combination of 10 and 20-year Linear is: 0.5148952267406232


In [93]:
print(f"R2 of SV+BV prediction using 10-year Linear only is: {r2_sv_bv_ind['Linear 10Yr']}")
print(f"R2 of SV+BV prediction using 20-year Linear only is: {r2_sv_bv_ind['Linear 20Yr']}")
print(f"R2 of SV+BV prediction using a combination of 10 and 20-year Linear is: {r2_sv_bv_comb}")

R2 of SV+BV prediction using 10-year Linear only is: 0.8596279440080496
R2 of SV+BV prediction using 20-year Linear only is: 0.8729709065232263
R2 of SV+BV prediction using a combination of 10 and 20-year Linear is: 0.8798022589790662


In [94]:
print(f"R2 of SG+BG prediction using 10-year Linear only is: {r2_sg_bg_ind['Linear 10Yr']}")
print(f"R2 of SG+BG prediction using 20-year Linear only is: {r2_sg_bg_ind['Linear 20Yr']}")
print(f"R2 of SG+BG prediction using a combination of 10 and 20-year Linear is: {r2_sg_bg_comb}")

R2 of SG+BG prediction using 10-year Linear only is: 0.9148920812521876
R2 of SG+BG prediction using 20-year Linear only is: 0.9242182926802163
R2 of SG+BG prediction using a combination of 10 and 20-year Linearo is: 0.9286029801367489


In [57]:
print(f"R2 of MOM prediction using 20-year Lasso only is: {twenty_r2_mom_lasso}")
print(f"R2 of MOM prediction using 10-year Lasso only is: {ten_r2_mom_lasso}")
print(f"R2 of MOM prediction using a combination of 10 and 20 year Lasso is: {r2_mom_comb}")

SSE of MOM prediction using 5-year Linear only is: 158821.98274577712
SSE of MOM prediction using 10-year Lasso only is: 150895.3941
SSE of MOM prediction using a combination of Linear and Lasso is: 153229.59724378915


In [59]:
print(f"SSE of SH+BH prediction using 5-year Linear only is: {sse_sh_bh_ind['Linear']}")
print(f"SSE of SH+BH prediction using 10-year Lasso only is: {sse_sh_bh_ind['Lasso']}")
print(f"SSE of SH+BH prediction using a combination of Linear and Lasso is: {sse_sh_bh_comb}")

SSE of SH+BH prediction using 5-year Linear only is: 29226.65642584544
SSE of SH+BH prediction using 10-year Lasso only is: 102523.974
SSE of SH+BH prediction using a combination of Linear and Lasso is: 44040.48280923926


In [60]:
print(f"SSE of SL+BL prediction using 5-year Linear only is: {sse_sl_bl_ind['Linear']}")
print(f"SSE of SL+BL prediction using 10-year Lasso only is: {sse_sl_bl_ind['Lasso']}")
print(f"SSE of SL+BL prediction using a combination of Linear and Lasso is: {sse_sl_bl_comb}")

SSE of SL+BL prediction using 5-year Linear only is: 40207.13960897462
SSE of SL+BL prediction using 10-year Lasso only is: 137581.7978
SSE of SL+BL prediction using a combination of Linear and Lasso is: 63481.32646771954
